In [16]:
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
import time
import logging
from tqdm import tqdm
import re

In [11]:
import os
os.chdir(r'/Users/lucyjhampton/Library/Mobile Documents/com~apple~CloudDocs/Jupyter/AI Bib')

Get data

In [13]:
papers_topics = pd.read_csv('papers_topics.csv', index_col = 0)
papers_topics

,Abstract,Topic,Probability,DOI,GPT_response,GPT_class,Input_tokens,Output_tokens,reason_stop,system_fingerprint,Publication Year,Cited by
0,Artificial Intelligence (AI)-driven language m...,8,0.897472,10.1057/s41599-024-02720-3,This paper assesses the global consequences of...,A,3948,50,stop,fp_71fab63729,2024,0.0
1,This paper aims to estimate the returns to sol...,2,0.843163,10.1016/j.eneco.2024.107768,This paper estimates the economic impact of so...,A,3889,60,stop,fp_71fab63729,2024,0.0
2,Fostering innovation is one of the key roles o...,-1,0.000000,10.1016/j.jeca.2024.e00372,This paper investigates the impact of investme...,A,3938,68,stop,fp_71fab63729,2024,0.0
3,Employment is a pivotal driver for ensuring an...,6,0.957130,10.1057/s41599-024-03333-6,This paper examines the impact of robot applic...,A,3955,59,stop,fp_71fab63729,2024,0.0
4,"In the current era, the symbiotic relationship...",6,0.988840,10.1057/s41599-024-02642-0,This paper explores the relationship between A...,A,3871,63,stop,fp_71fab63729,2024,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4980,The study examines the intention of stock inve...,7,0.586814,10.1016/j.bir.2023.09.005,This paper investigates stock investors' inten...,A,3897,57,stop,fp_71fab63729,2023,3.0
4981,Recent robotics and artificial intelligence ad...,6,0.837150,10.1093/ser/mwae025,This paper investigates the relationship betwe...,A,3871,58,stop,fp_71fab63729,2024,0.0
4982,While algorithmic management provides new opti...,9,1.000000,10.1016/j.tourman.2022.104716,This paper explores the impact of algorithmic ...,A,3890,69,stop,fp_71fab63729,2023,12.0
4983,In order to investigate the acceptance of a Pa...,20,1.000000,10.1016/j.tra.2022.01.009,This paper investigates the acceptance of a Pa...,A,3968,63,stop,fp_71fab63729,2022,10.0


Write prompt

In [23]:
prompt = "You are an academic expert in economics, finance, management and innovation studies. You are provided with abstracts from academic papers. Your goal is to carefully review each abstract to identify the primary focus and methodology of the paper. Based on this analysis, classify each paper into one of the following categories. \n\nCategory A: The paper studies: \n- The impact of AI on economic (including but not limited to employment, growth, currency, productivity, competition, economic welfare) or management/innovation (such as corporate governance, business practices, innovation and employee retention) outcomes. Note that these outcomes do not have to be macroeconomic, e.g., the paper can focus on a specific firm's productivity or financial decisions. \n- The measurement of the economic impact of AI, including measurement challenges. \n- AI’s implications for ethics, the environment, or policy; provided they focus on how these issues relate to economic activity, e.g., the relationship between AI, racial inequality and income inequality. \n- The adoption of AI by firms, the government, or sectors; what affects adoption decisions; and how AI is used. \n- Economic drivers of AI progress, such as cost of compute. \nProvided they meet the earlier requirements, include both theoretical and empirical, micro and macro studies from the fields of economics, management, finance, and innovation (including sector-specific studies). \n\nCategory B: The paper does not fit into category A. This includes papers that develop or apply a machine learning technique, or measure to study an outcome other than the impact of AI, and papers that develop an AI tool. This includes papers that focus on developing machine learning techniques to predict financial returns. It also includes papers that focus on social outcomes such as race, religion and politics where the paper does not make explicit links to economic outcomes, such as economic inequality, unemployment or resource distribution. Include in this category papers that are not about AI. \n\nInstructions: 1. Reason first, and summarise in no more than three bullet points, the main focus and methodology of the paper outlined in the abstract. 2. Only when you have enough information, decide on a classification based on 1. Do not use more than 100 words in your response. When giving your final classification, format it as 'Class: x' where x is your suggested classification. For each abstract you correctly classify, I will pay you $10 as a tip."

In [24]:
print(prompt)

You are an academic expert in economics, finance, management and innovation studies. You are provided with abstracts from academic papers. Your goal is to carefully review each abstract to identify the primary focus and methodology of the paper. Based on this analysis, classify each paper into one of the following categories. 

Category A: The paper studies: 
- The impact of AI on economic (including but not limited to employment, growth, currency, productivity, competition, economic welfare) or management/innovation (such as corporate governance, business practices, innovation and employee retention) outcomes. Note that these outcomes do not have to be macroeconomic, e.g., the paper can focus on a specific firm's productivity or financial decisions. 
- The measurement of the economic impact of AI, including measurement challenges. 
- AI’s implications for ethics, the environment, or policy; provided they focus on how these issues relate to economic activity, e.g., the relationship bet

Topic 0: Supply Chains

In [25]:
topic_0 = papers_topics[papers_topics['Topic'] == 0]
topic_0 = topic_0[['Abstract', 'Topic', 'DOI']]
topic_0

,Abstract,Topic,DOI
6,The technology of the Internet of Things (IoT)...,0,10.1186/s41072-024-00177-w
14,The auto parts manufacturing sector faces mult...,0,10.1016/j.seps.2024.101908
25,To achieve the global carbon neutrality goal b...,0,10.1016/j.ijpe.2024.109286
28,Global carbon reduction drives the transformat...,0,10.1016/j.resconrec.2024.107671
40,Despite the fact that artificial intelligence ...,0,10.1016/j.strueco.2024.05.010
...,...,...,...
4949,Predictive maintenance is a concept linked to ...,0,10.2495/EPM220111
4950,Meituan pioneered an on -demand food delivery ...,0,10.1287/inte.2023.0084
4953,This article explores the impact of the smart ...,0,10.30525/2256-0742/2023-9-3-214-221
4972,The article considers the problem of increasin...,0,10.1007/978-3-030-40749-0_52


In [26]:
example1 = """Production rates are considered as an essential aspect of construction industry because they are the indicators of the productivity efficiency of construction sector. However, there is a gap in identifying factors affecting rebar workers and their production rate. This study tries to bridge this gap by developing a neural network model for estimating rebar labourâ€™s production rates. A questionnaire has been distributed and statistical software program was used to analyze the collected data. Two methods were used for analysis. The first depends on relative importance index, whereas the second depends on the probability and the impact. The results indicate that â€˜project typeâ€™ is the most important factor affecting the productivity. A model was estimated to predict rebar laboursâ€™ productivity. Reliable values have been successfully predicted by artificial neural network. This article presents a software program, which is used to measure the production rate (Output) based on the data provided in the form of factors affecting the rebar labour (Input). This helps to measure productivity growth in a low-cost residential building. In addition, it supports fundamentals building by predicting the production rate of rebar labour, to establish a database for executed projects in the future to develop productivity estimation process."""
output1 = """This study develops a neural network model to estimate the production rates of rebar workers in the construction industry, addressing a gap in identifying factors that affect productivity. It utilizes data analysis to determine that project type is the most significant factor influencing productivity. The resulting model aims to predict rebar labor productivity and support future project estimations. Since the study uses AI as a tool to analyze and predict productivity rates, rather than examining the broader economic implications of AI itself, it should be classified as Class B. Class: B"""

example2 = """New trends in the management of production costs are associated with the development of Machine learning, taking into account the human factor. They often use a bifurcation procedure that assigns costs to one of two categories. For this, training of pattern recognition is applied. Systems engineering allows for more sophisticated partition learning models that categorize costs into one of four categories (quartering). For this, two bifurcation training models are integrated. Such quartering is used by the management of the company to stimulate personnel responsible for costs. However, if the management is not aware of the minimum costs, undesirable activity of the personnel is possible. In this case, personnel, as an active element, chooses costs so as to maximize own incentives. For this reason, the real costs of the company may be higher than the minimum. Therefore, traditional machine supervised learning algorithms for pattern recognition can be ineffective. To solve this problem in the face of uncertainty, a mechanism has been proposed that includes a procedure for training of quartering with the help of instructions from a trainer, as well as a procedure for stimulating. Sufficient conditions for the synthesis of such a mechanism are found, under which the costs are minimal. The use of this mechanism is illustrated with the example of training of quartering and incentives to reduce the cost of re-equipment of locomotives."""
output2 = """This paper explores the integration of machine learning in managing production costs, focusing on how it can help categorize costs using bifurcation and quartering procedures. While the study involves AI and machine learning as tools to improve cost management, it does not focus on the economic implications of AI itself. Therefore, it should be classified as Class B. Class: B"""

example3 = """Logistics in companies is a necessary process that has high costs with mostly no added value. Lowering this cost is vitally important for companies to stay competitive. Nowadays, storage systems are a critical part of any company's logistic system, and many of them try to reach an optimum level where they can operate with little freedom of movement of goods declared by the changing market. There are several manual and automated methods to achieve this. However, we hear quite little about the use of artificial intelligence in the field. This study focuses on the implementation of AI technology into warehousing, especially in categorizing goods. After an overview of the recent literature on AI technologies and their application in the field of logistics, the introduction of an AI application follows. The main goal of the application is to categorize each good stored in a warehouse into ABC-XYZ groups, which determines the place of the good in the warehouse and the ordering frequency with the quantity. After acquiring and cleaning the training data from a real company, the determination and selection of the least input parameters is an important and challenging task, which is demonstrated. The effectiveness of the supervised learning can be seen as an ANN (artificial neural network) can output, with the aid of a non-conventional metaheuristic approach - the black hole algorithm - as the learning agent is demonstrated by an example, which also shows the result of an ABC-XYZ categorization run on a dataset from a multinational company."""
output3 = """This paper focuses on implementing AI technology in warehousing logistics, specifically for categorizing goods. The study addresses the adoption of AI by firms and its implications for operational efficiency, which qualifies it for category A. Class: A"""

example4 = """Purpose: The present paper aims to demonstrate the potential of integration between industrial robotics and Lean Manufacturing (LM) approach to increase the efficiency of an assembly line. Design/methodology/approach: Based on a case study performed in an Italian company, this paper reports a comparative analysis of the results produced on a line balancing study involving a semi-automated production line, aided by an industrial robot. Findings: The results suggest the possibility of implementing industrial robotics in line balancing studies highlighting efficiency gains and idle reduction. Further, it also addresses some concepts directly related to industry 4.0, such as collaborative robotics, artificial intelligence, and lean automation. Practical implications: Line balancing studies may include advanced robotics in order to extend traditional lean practices toward Digital LM. Originality/value: This study adds contributions to the operational excellence literature, demonstrating the symbiosis between industrial robotics and LM practices."""
output4 = """This paper investigates the integration of industrial robotics with Lean Manufacturing to enhance assembly line efficiency. It employs a case study methodology to analyze the impact of robotics on line balancing. The focus on operational efficiency and the implications for management practices qualifies it for category A. Class: A"""


In [28]:
## PROMPT GPT-4o

# set up logging
logging.basicConfig(level = logging.INFO)

# load data
classed_0s = pd.DataFrame(columns=['Abstract', 'Topic', 'DOI', 'GPT_response', 'GPT_class', 'Input_tokens', 'Output_tokens', 'reason_stop', 'system_fingerprint'])
processed_DOIs = set()
data_dict = {
    'Abstract': [],
    'Topic': [],
    'DOI': [],
    'GPT_response': [],
    'GPT_class': [],
    'Input_tokens': [],
    'Output_tokens': [],
    'reason_stop': [],
    'system_fingerprint': []
}

# set up api
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()
client.api_key = api_key

# loop through abstracts
for index, row in tqdm(topic_0.iterrows(), desc = 'Processing abstracts'):
    if row['DOI'] in processed_DOIs:
        continue
        
    else:
        if pd.isnull(row['Abstract']):
            print('blank!')
            data_dict['Abstract'].append(row['Abstract'])
            data_dict['Topic'].append(row['Topic'])
            data_dict['DOI'].append(row['DOI'])
            data_dict['GPT_response'].append('')
            data_dict['GPT_class'].append('')
            data_dict['Input_tokens'].append('')
            data_dict['Output_tokens'].append('')
            data_dict['reason_stop'].append('')
            data_dict['system_fingerprint'].append('')
        else:
            response = client.chat.completions.create(
                model="gpt-4o",
                max_tokens=300,
                temperature=0,
                seed=2345,
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": example1},
                    {"role": "assistant", "content": output1},
                    {"role": "user", "content": example2},
                    {"role": "assistant", "content": output2},
                    {"role": "user", "content": example3}, 
                    {"role": "assistant", "content": output3},
                    {"role": "user", "content": example4},
                    {"role": "assistant", "content": output4},
                    {"role": "user", "content": row['Abstract']}
                ]
            )
        
            # store data in df
            data_dict['Abstract'].append(row['Abstract'])
            data_dict['Topic'].append(row['Topic'])
            data_dict['DOI'].append(row['DOI'])
            message = response.choices[0].message.content
            data_dict['GPT_response'].append(message)
            try:
                data_dict['GPT_class'].append(re.search(r'Class:\s*([A-Z])', message).group(1))
            except ValueError:
                data_dict['GPT_class'].append('')
            data_dict['Input_tokens'].append(response.usage.prompt_tokens)
            data_dict['Output_tokens'].append(response.usage.completion_tokens)
            data_dict['reason_stop'].append(response.choices[0].finish_reason)
            data_dict['system_fingerprint'].append(response.system_fingerprint)
    
        # create df out of dict
        classed_0s = pd.DataFrame(data_dict)
        
        # add to list of processed urls
        processed_DOIs.add(row['DOI'])

    # # sleep to observe rate limit (500 requests per minute)
    # time.sleep(0.01)

# Display the DataFrame
classed_0s = pd.DataFrame(data_dict)
classed_0s.to_csv("classed_0s.csv", index=False)
classed_0s

Processing abstracts: 0it [00:00, ?it/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing abstracts: 1it [00:03,  3.08s/it]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing abstracts: 2it [00:04,  2.38s/it]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing abstracts: 3it [00:07,  2.57s/it]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing abstracts: 4it [00:09,  2.25s/it]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing abstracts: 5it [00:11,  2.27s/it]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing abstracts: 6it [00:13,  2.07s/it]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing abstracts: 7it [00:15,  1.93s/it]INFO:httpx:HTTP

,Abstract,Topic,DOI,GPT_response,GPT_class,Input_tokens,Output_tokens,reason_stop,system_fingerprint
0,The technology of the Internet of Things (IoT)...,0,10.1186/s41072-024-00177-w,This paper uses machine learning to analyze th...,A,1949,74,stop,fp_a7d06e42a7
1,The auto parts manufacturing sector faces mult...,0,10.1016/j.seps.2024.101908,This study presents an integrated data-driven ...,A,1979,77,stop,fp_a7d06e42a7
2,To achieve the global carbon neutrality goal b...,0,10.1016/j.ijpe.2024.109286,This study examines the relationship between A...,A,2002,94,stop,fp_a7d06e42a7
3,Global carbon reduction drives the transformat...,0,10.1016/j.resconrec.2024.107671,This paper develops an intelligent green suppl...,A,1949,64,stop,fp_a7d06e42a7
4,Despite the fact that artificial intelligence ...,0,10.1016/j.strueco.2024.05.010,This paper examines the role of AI in enhancin...,A,1998,83,stop,fp_a7d06e42a7
...,...,...,...,...,...,...,...,...,...
625,Predictive maintenance is a concept linked to ...,0,10.2495/EPM220111,This paper explores a predictive maintenance s...,A,1994,65,stop,fp_a7d06e42a7
626,Meituan pioneered an on -demand food delivery ...,0,10.1287/inte.2023.0084,This paper discusses Meituan's development of ...,A,2019,64,stop,fp_a7d06e42a7
627,This article explores the impact of the smart ...,0,10.30525/2256-0742/2023-9-3-214-221,This article examines the impact of the smart ...,A,2195,96,stop,fp_a7d06e42a7
628,The article considers the problem of increasin...,0,10.1007/978-3-030-40749-0_52,This paper addresses enhancing the competitive...,A,1946,68,stop,fp_a7d06e42a7


Topic 2: Energy and Sustainability

In [5]:
example1 = """The fourth industrial revolution has brought about key technological breakthroughs represented by artificial intelligence (AI) and blockchain (BC), which provide an opportunity for the development of clean energy (CE). We employ a wavelet-based quantile-on-quantile approach to investigate the correlation among AI, BC and CE over different periods and time horizons. The results show that although the integration of AI and CE systems requires a lot of time, it can significantly boost CE development in the long run. However, BC cannot significantly promote CE development except for a few extreme periods. Our results are supported by the mechanism of interaction among AI, BC and CE development, which highlights their relationship. Therefore, the government should focus on promoting the development of new information technologies and their integration with the CE system, which is crucial for the implementation of energy transition strategies. In addition, low-carbon investors should focus on these technologies because of their relationship with CE. ¬© 2023"""
output1 = """This paper investigates the relationship between AI, blockchain, and clean energy development using a quantile-on-quantile approach. It discusses the long-term economic impacts of AI on clean energy systems and provides policy recommendations. Class: A""" 

example3 = """This paper explores the intersection between artificial intelligence and nuclear energy, shedding light on the intriguing scenario when these two sectors jointly consturct. Through the application of both full-sample and sub-sample methodologies, this study identifies the time-dependent interrelationships between China's artificial intelligence index (AI) and nuclear energy indicator (NUC). The quantitative analysis presents that AI's influence on nuclear energy is twofold. On one hand, AI contributes positively by acting as a catalyst and enhancing safety measures in the nuclear sector. On the other, the impact might be perceived negatively, primarily when cost-effective alternative energy sources overshadow the benefits of nuclear energy. Additionally, the positive effect of NUC on AI highlights the benefits derived from nuclear's expansive and consistent energy output, catering efficiently to AI's substantial energy demands. In essence, AI and NUC are found to be complementary, with each having the potential to propel the other forward. This reciprocity paves the way for a synergistic relationship, promising mutual benefits. The study introduces a fresh perspective on the co-evolution of energy and technology, offering thought-provoking recommendations aimed at cultivating the collaborative growth of AI and NUC towards a common good. ¬© 2023"""
output3 = """This paper examines the relationship between AI and nuclear energy, focusing on their interdependencies and mutual benefits. It employs quantitative analysis to assess how AI enhances safety in nuclear energy while also considering the competitive landscape of energy sources. It investigates the implications of AI for economic outcomes within the nuclear energy sector, so it is relevant. Class: A"""

example2 = """Greenwashing is the misleading use of environmental claims to market non-eco-friendly products, a growing concern as environmentally conscious consumers seek eco-friendly options. This study aims to identify factors that allow the detection of ‚Äúgreenwashing‚Äù practices in Colombian companies across various industrial sectors holding certifications such as ISO14001, the Colombian environmental seal, the district environmental excellence program, or international/regional environmental certifications. We employed a quantitative, cross-sectional methodology using three machine learning algorithms (Decision Tree Classification, Random Forest, Logistic Regression) and the double machine learning technique to identify causal effects and significant variables. Additionally, a greenwashing indicator was formulated, considering two variables: (a) Presence of a comprehensive solid waste management plan, and (b) Presence of a water usage and conservation program. Our findings reveal substantial variations in the adoption of environmental management practices across diverse industrial sectors and regions in Colombia. It is noteworthy that our research emphasizes that: (i) companies with an environmental contingency plan and a monitoring program are less likely to engage in greenwashing; (ii) expenses for personnel dedicated to environmental protection activities and the investment and expenses related to air and climate protection also reduce the likelihood of greenwashing; (iii) companies in the manufacturing of non-metallic mineral products sector are more likely to engage in greenwashing. It was also found that a significant proportion of Colombian industrial companies with environmental certifications do not fully embrace sustainable practices, implying limited effectiveness of environmental certifications in the country (56,09%). This research contributes to the taxonomy of greenwashing by delving into various types identified in the literature. It sheds light on selective disclosure, emphasizing that mere certification without tangible actions can constitute greenwashing. The broadened concept of decoupling now includes not only meeting shareholder expectations but also failing to meet those set by certifications. Attention deflection involves diverting focus toward certifications rather than real investments in environmental practices. Deceptive manipulation occurs when certifications create a misleading perception without concrete actions. Additionally, dubious authorizations and labels are revealed, showcasing the vulnerability of eco-labels to fraud. The study enriches existing categories, viewing certifications as potential instruments for greenwashing, reinforcing the need for substantial, measurable actions to validate sustainability claims. ¬© 2024 The Author(s)"""
output2 = """This paper investigates greenwashing practices in Colombian companies using machine learning algorithms to identify factors that contribute to misleading environmental claims. The paper does not address the impact or implications of AI. Class: B"""

example5 = """In the context of the¬†rapid development of artificial intelligence, industrial robots, as an important manifestation of artificial intelligence technology application, provide enterprises with a ‚Äúcreative destruction‚Äù environment, and play a positive role in energy utilization and environmental governance. Therefore, academics have focused a lot of attention on the question of whether industrial robot application can successfully improve corporate green innovation (CGI), thus achieving green transformation, and upgrading the manufacturing industry. This paper builds a ‚ÄúBartik instrument variable‚Äù to calculate the micro-enterprise robot penetration rate from 2007 to 2019, looks at the effect of industrial robot application on CGI using data from manufacturing listed companies, and discusses the mechanism, heterogeneity, and robustness of this effect. The research finds that industrial robot application significantly improve CGI, indicating that they can promote corporate green technology transformation. This effect not only enhances the scale of CGI but also strengthens the quality of CGI. In addition, the above conclusion still holds after a series of endogeneity and robustness tests. Mechanism research shows that industrial robot application contribute to CGI by improving production efficiency, human capital, and environmental governance levels. It has a good complementary optimization effect on CGI. Furthermore, heterogeneity research shows that industrial robot application has a more significant role in enhancing green innovation for enterprises that are labor-intensive, more intense market competitive, more serious environmental pollution, and have lower level of digital economy development. The research conclusion verifies the effect of artificial intelligence technology and explores its mechanism, which provides empirical references for improving corporate environmental performance and green transformation. ¬© The Author(s), under exclusive licence to Springer Nature B.V. 2023."""
output5 = """This paper investigates the impact of industrial robots, as a manifestation of AI, on corporate green innovation (CGI) in manufacturing. It uses a Bartik instrument variable to analyze data from listed companies, exploring mechanisms and heterogeneity in the effects. The findings indicate that industrial robots significantly enhance CGI, linking AI technology to environmental performance and corporate transformation. Class: A."""

example4 = """Using firm-level data in China from 2011‚Äì2022, we employ text analysis and machine learning methods to construct government environmental initiatives (GEI) index, and examine the impact of GEI on corporate environment, social, and governance (ESG) performance. Based on attention-based view (ABV), we find that GEI increases corporate ESG performance by promoting managerial environmental attention (MEA), especially in high-polluting industries and state-owned enterprises (SOEs). Furthermore, the results remain unchanged after robustness tests. Our findings imply that GEI serves as an effective means for advancing the sustainable development of listed companies. ¬© 2024 Informa UK Limited, trading as Taylor & Francis Group."""
output4 = """This paper employs text analysis and machine learning to construct an index of government environmental initiatives and examines their impact on corporate ESG performance. It does not investigate the impact/implications of ML in economic outcomes, it just uses ML techniques. Class: B."""

In [ ]:
## PROMPT GPT-4o mini again

from openai import OpenAI
from dotenv import load_dotenv
import time
import logging
from tqdm import tqdm
import re

# set up logging
logging.basicConfig(level = logging.INFO)

# Load progress from the last run (if any)
try:
    classed_As = pd.read_csv("classed_As.csv")
    processed_DOIs = set(classed_As['DOI'])
    data_dict = classed_As.to_dict('list')
except FileNotFoundError:
    classed_As = pd.DataFrame(columns=['Abstract', 'DOI', 'GPT_response', 'GPT_class', 'Input_tokens', 'Output_tokens', 'reason_stop', 'system_fingerprint'])
    processed_DOIs = set()
    data_dict = {
        'Abstract': [],
        'DOI': [],
        'GPT_response': [],
        'GPT_class': [],
        'Input_tokens': [],
        'Output_tokens': [],
        'reason_stop': [],
        'system_fingerprint': []
    }

# set up api
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()
client.api_key = api_key

# loop through abstracts
for index, row in tqdm(As.iterrows(), desc = 'Processing abstracts'):
    if row['DOI'] in processed_DOIs:
        continue
        
    else:
        if pd.isnull(row['Abstract']):
            print('blank!')
            data_dict['Abstract'].append(row['Abstract'])
            data_dict['DOI'].append(row['DOI'])
            data_dict['GPT_response'].append('')
            data_dict['GPT_class'].append('')
            data_dict['Input_tokens'].append('')
            data_dict['Output_tokens'].append('')
            data_dict['reason_stop'].append('')
            data_dict['system_fingerprint'].append('')
        else:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                max_tokens=300,
                temperature=0,
                seed=2345,
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": example1},
                    {"role": "assistant", "content": output1},
                    {"role": "user", "content": example2},
                    {"role": "assistant", "content": output2},
                    {"role": "user", "content": example3}, 
                    {"role": "assistant", "content": output3},
                    {"role": "user", "content": example4},
                    {"role": "assistant", "content": output4},
                    {"role": "user", "content": example5},
                    {"role": "assistant", "content": output5},
                    {"role": "user", "content": example6},
                    {"role": "assistant", "content": output6},
                    {"role": "user", "content": example7},
                    {"role": "assistant", "content": output7},
                    {"role": "user", "content": example8},
                    {"role": "assistant", "content": output8},
                    {"role": "user", "content": example9},
                    {"role": "assistant", "content": output9},
                    {"role": "user", "content": example10},
                    {"role": "assistant", "content": output10},
                    {"role": "user", "content": example11},
                    {"role": "assistant", "content": output11},
                    {"role": "user", "content": example12},
                    {"role": "assistant", "content": output12},
                    {"role": "user", "content": example13},
                    {"role": "assistant", "content": output13},
                    {"role": "user", "content": example14},
                    {"role": "assistant", "content": output14},
                    {"role": "user", "content": row['Abstract']}
                ]
            )
        
            # store data in df
            data_dict['Abstract'].append(row['Abstract'])
            data_dict['DOI'].append(row['DOI'])
            message = response.choices[0].message.content
            data_dict['GPT_response'].append(message)
            try:
                data_dict['GPT_class'].append(re.search(r'Class:\s*([A-Z])', message).group(1))
            except ValueError:
                data_dict['GPT_class'].append('')
            data_dict['Input_tokens'].append(response.usage.prompt_tokens)
            data_dict['Output_tokens'].append(response.usage.completion_tokens)
            data_dict['reason_stop'].append(response.choices[0].finish_reason)
            data_dict['system_fingerprint'].append(response.system_fingerprint)
    
        # create df out of dict
        classed_As = pd.DataFrame(data_dict)
        
        # save progress, add url to processed url list
        if (index + 1) % 10 == 0:
            try:
                if len(classed_As) - len(pd.read_csv("classed_As.csv")) == 10:
                    new_papers = classed_As.tail(10)
                    new_papers.to_csv("classed_As.csv", index=False, mode='a', header=False)
                    logging.info("Saved progress at iteration {}".format(index + 1))
                else:
                    logging.info("Warning: trying to overwrite with blank at iteration {}".format(index + 1))
                    break
            except FileNotFoundError:
                classed_As.to_csv("classed_As.csv", index=False, header = True)
        
        # add to list of processed urls
        processed_DOIs.add(row['DOI'])

    # # sleep to observe rate limit (500 requests per minute)
    # time.sleep(0.01)

# Display the DataFrame
classed_As = pd.DataFrame(data_dict)
classed_As.to_csv("classed_As.csv", index=False)
classed_As

Topic 3: Health

In [1]:
example1 = """Various factors are behind the forces that drive hospitals toward more sustainable operations. Hospitals contracting with Medicare, for instance, are reimbursed for the procedures performed, regardless of the number of days that patients stay in the hospital. This reimbursement structure has incentivized hospitals to use their resources (such as their beds) more efficiently to maximize revenues. One way hospitals can improve bed utilization is by predicting patientsâ€™ length of stay (LOS) at the time of admission, the benefits of which extend to employees, communities, and the patients themselves. In this paper, we employ a data analytics approach to develop and test a deep learning neural network to predict LOS for patients with chronic obstructive pulmonary disease (COPD) and pneumonia. The theoretical contribution of our effort is that it identifies variables related to patientsâ€™ prior admissions as important factors in the prediction of LOS in hospitals, thereby revising the current paradigm in which patientsâ€™ medical histories are rarely considered for the prediction of LOS. The methodological contributions of our work include the development of a data engineering methodology to augment the data sets, prediction of LOS as a numerical (rather than a binary) variable, temporal evaluation of the training and validation data sets, and a significant improvement in the accuracy of predicting LOS for COPD and pneumonia inpatients. Our evaluations show that variables related to patientsâ€™ previous admissions are the main driver of the deep network's superior performance in predicting the LOS as a numerical variable. Using the assessment criteria introduced in prior studies (i.e., Â±2 days and Â±3 days tolerance), our models are able to predict the length of hospital stay with 86 % and 91 % accuracy for the COPD data set, and with 74 % and 85 % accuracy for the pneumonia data set. Hence, our effort could help hospitals serve a larger number of patients with a fixed amount of resources, thereby reducing their environmental footprint while increasing their revenue, as well as their patientsâ€™ satisfaction."""
output1 = """This study investigates the factors driving hospitals to more sustainable operations. While this study uses AI (deep learning) to achieve its goals, AI is not the subject of the study but merely the technique used. Class: B"""

example2 = """With a population of more than 1.3 billion people, India has a rapidly growing need for health care, particularly in rural areas. Along with a widespread increase in the incidence of cancer, diabetes, and cardiovascular disease, the nation also is suffering from a shortage of health-care professionals. At Apollo Hospitals Enterprise Limited, the use of robotic surgery and artificial intelligence (AI) has allowed medical practitioners to provide more personalized health care to patients, increased the efficiency of doctors by removing repetitive tasks, and facilitated a shift toward prevention of serious illnesses. An analysis of the opportunities and challenges posed by the technologies employed at Apollo Hospitals' facilities highlights the promise of AI in the health-care field and the obstacles yet to be overcome."""
output2 = """This paper examines the use of AI and robotic surgery in healthcare at Apollo Hospitals in India. It discusses the impact of these technologies on healthcare delivery, efficiency, and patient care, focusing on management outcomes. Class: A"""

example3 = """Use of Artificial Intelligence (AI) in variety of areas has encouraged an extensive global discourse on the underlying ethical principles and values. With the rapid AI development process and its near instant global coverage, the issues of applicable ethical principles and guidelines have become vital. AI promises to deliver a lot of advantages to economic, social and educational fields. Since AI is also increasingly applied in healthcare and medical education areas, ethical application issues are growing ever more important. Ethical and social issues raised by AI in healthcare overlap with those raised by personal data use, function automation, reliance on assistive medical technologies and the so-called â€˜telehealthâ€™. Without well-grounded ethical guidelines or even regulatory framework in respect of the AI in healthcare several legal and ethical problems at the implementational level can arise. In order to facilitate further discussion about the ethical principles and responsibilities of educational system in healthcare using AI and to potentially arrive at a consensus concerning safe and desirable uses of AI in healthcare education, this paper performs an evaluation of the self-imposed AI ethical guidelines identifying the common principles and approaches as well as drawbacks limiting the practical and legal application of internal policies. The main aim of the research is to encourage integration of theoretical studies and policy studies on sustainability issues in correlation between healthcare and technologies, the AI ethical perspective."""
output3 = """This paper evaluates ethical principles and guidelines for AI in healthcare, focusing on the implications of AI's application in this sector. The paper's focus on ethical implications related to economic and social outcomes of AI qualifies it for category A. Class: A"""

example4 = """Texas is the only state that does not mandate that employers carry workersâ€™ compensation (WC) insurance coverage. In place of traditional WC, companies can choose to offer alternative â€œnon-subscriptionâ€ disability plans to workers. Little large-scale empirical research has studied the consequences of switching from traditional WC to non-subscription plans. We use a difference-in-differences estimator along with a novel machine learning approach to compare effects of switching to non-subscription plans for employees in Texas versus contemporaneously measured non-Texas-based employees for 25 large companies. Our results indicate that total medical payments dropped by roughly 40% from switching to non-subscription plans, consisting of reductions in hospital spending, physician spending, and other medical spending. Similarly, indemnity payments dropped by 70% and number of reported lost days dropped by 80%. Accumulating all sources of spending, we find that total expense associated with workplace injury fell by approximately 46% after switching to the non-subscription program."""
output4 = """Although machine learning is employed in the study, the primary focus is on the economic impact of different insurance models (WC vs. non-subscription plans) and the associated cost reductions, rather than on the role of AI itself. Here, machine learning is a tool for analyzing the outcomes, not the subject of the research, therefore it fits into class B. Class: B"""

In [ ]:
## PROMPT GPT-4o mini again

from openai import OpenAI
from dotenv import load_dotenv
import time
import logging
from tqdm import tqdm
import re

# set up logging
logging.basicConfig(level = logging.INFO)

# Load progress from the last run (if any)
try:
    classed_As = pd.read_csv("classed_As.csv")
    processed_DOIs = set(classed_As['DOI'])
    data_dict = classed_As.to_dict('list')
except FileNotFoundError:
    classed_As = pd.DataFrame(columns=['Abstract', 'DOI', 'GPT_response', 'GPT_class', 'Input_tokens', 'Output_tokens', 'reason_stop', 'system_fingerprint'])
    processed_DOIs = set()
    data_dict = {
        'Abstract': [],
        'DOI': [],
        'GPT_response': [],
        'GPT_class': [],
        'Input_tokens': [],
        'Output_tokens': [],
        'reason_stop': [],
        'system_fingerprint': []
    }

# set up api
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()
client.api_key = api_key

# loop through abstracts
for index, row in tqdm(As.iterrows(), desc = 'Processing abstracts'):
    if row['DOI'] in processed_DOIs:
        continue
        
    else:
        if pd.isnull(row['Abstract']):
            print('blank!')
            data_dict['Abstract'].append(row['Abstract'])
            data_dict['DOI'].append(row['DOI'])
            data_dict['GPT_response'].append('')
            data_dict['GPT_class'].append('')
            data_dict['Input_tokens'].append('')
            data_dict['Output_tokens'].append('')
            data_dict['reason_stop'].append('')
            data_dict['system_fingerprint'].append('')
        else:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                max_tokens=300,
                temperature=0,
                seed=2345,
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": example1},
                    {"role": "assistant", "content": output1},
                    {"role": "user", "content": example2},
                    {"role": "assistant", "content": output2},
                    {"role": "user", "content": example3}, 
                    {"role": "assistant", "content": output3},
                    {"role": "user", "content": example4},
                    {"role": "assistant", "content": output4},
                    {"role": "user", "content": example5},
                    {"role": "assistant", "content": output5},
                    {"role": "user", "content": example6},
                    {"role": "assistant", "content": output6},
                    {"role": "user", "content": example7},
                    {"role": "assistant", "content": output7},
                    {"role": "user", "content": example8},
                    {"role": "assistant", "content": output8},
                    {"role": "user", "content": example9},
                    {"role": "assistant", "content": output9},
                    {"role": "user", "content": example10},
                    {"role": "assistant", "content": output10},
                    {"role": "user", "content": example11},
                    {"role": "assistant", "content": output11},
                    {"role": "user", "content": example12},
                    {"role": "assistant", "content": output12},
                    {"role": "user", "content": example13},
                    {"role": "assistant", "content": output13},
                    {"role": "user", "content": example14},
                    {"role": "assistant", "content": output14},
                    {"role": "user", "content": row['Abstract']}
                ]
            )
        
            # store data in df
            data_dict['Abstract'].append(row['Abstract'])
            data_dict['DOI'].append(row['DOI'])
            message = response.choices[0].message.content
            data_dict['GPT_response'].append(message)
            try:
                data_dict['GPT_class'].append(re.search(r'Class:\s*([A-Z])', message).group(1))
            except ValueError:
                data_dict['GPT_class'].append('')
            data_dict['Input_tokens'].append(response.usage.prompt_tokens)
            data_dict['Output_tokens'].append(response.usage.completion_tokens)
            data_dict['reason_stop'].append(response.choices[0].finish_reason)
            data_dict['system_fingerprint'].append(response.system_fingerprint)
    
        # create df out of dict
        classed_As = pd.DataFrame(data_dict)
        
        # save progress, add url to processed url list
        if (index + 1) % 10 == 0:
            try:
                if len(classed_As) - len(pd.read_csv("classed_As.csv")) == 10:
                    new_papers = classed_As.tail(10)
                    new_papers.to_csv("classed_As.csv", index=False, mode='a', header=False)
                    logging.info("Saved progress at iteration {}".format(index + 1))
                else:
                    logging.info("Warning: trying to overwrite with blank at iteration {}".format(index + 1))
                    break
            except FileNotFoundError:
                classed_As.to_csv("classed_As.csv", index=False, header = True)
        
        # add to list of processed urls
        processed_DOIs.add(row['DOI'])

    # # sleep to observe rate limit (500 requests per minute)
    # time.sleep(0.01)

# Display the DataFrame
classed_As = pd.DataFrame(data_dict)
classed_As.to_csv("classed_As.csv", index=False)
classed_As

Topic 6: Labour

In [7]:
example1 = """Based on a unique dataset on the use of artificial intelligence (AI) among employees in Denmark, we investigate within-job relationships between AI use and skill requirements. We show that the effects of AI are varied and depend on whether AI is used for providing orders to humans or providing information for further human handling and in which occupation it is used. AI may enhance or augment skills through, for example, the increased use of high-performance work practices, or it may increase work pace constraints and reduce employee autonomy. The results imply that the diffusion of AI can increase inequalities in the labour market by augmenting skills used in high-skill jobs, although having relatively more adverse impacts on other jobs. We use additive noise modelling to establish the likely direction of causality in our results and find that the direction of causality is from AI use to skill requirements."""
output1 = """This paper investigates the relationship between AI use and skill requirements in various occupations. It explores how AI can enhance or constrain skills, potentially increasing inequalities in the labor market. The study employs additive noise modeling to establish causality. This focus on the impact of AI on labor market outcomes qualifies it for category A. Class: A"""

example2 = """Purpose: The purpose of this paper is to investigate self-competence—the ability to act responsibly on one's own—and likely nonlinear wage returns across different levels of self-competence as part of training curricula. Design/methodology/approach: The authors identify the teaching of self-competence at the occupational level by applying machine-learning methods to the texts of occupational training curricula. Defining three levels of self-competence (high, medium, and low) and using individual labor market data, the authors examine nonlinearities in wage returns to different levels of self-competence. Findings: The authors find nonlinear returns to teaching self-competence: a medium level of self-competence taught in an occupation has the largest wage returns compared to low or high levels. However, in occupations with a high cognitive requirement profile, a high level of self-competence generates positive wage returns. Originality/value: This paper first adds to research on the importance of teaching noncognitive skills for economic outcomes, which recently—in addition to personality traits research—has primarily focused on social skills by introducing self-competence as another largely unexplored but important noncognitive skill. Second, the paper studies not only average but also nonlinear wage returns, showing that the right level of self-competence is crucial, i.e. neither teaching too little nor too much self-competence provides favorable returns because of trade-offs with other skills (e.g. technical or professional skills). Third, the paper also examines complementarities between cognitive skills and noncognitive skills, again pointing toward nonlinear returns, i.e. only in occupations with a high cognitive requirement profile, high levels of self-competence generate positive wage returns."""
output2 = """This paper investigates the impact of teaching self-competence on wage returns using machine learning methods. It finds nonlinear wage returns based on different levels of self-competence and examines the interaction between cognitive and noncognitive skills. While it involves machine learning, it only uses it as an econometric technique and does not focus on its economic impacts, so it is in category B. Class: B"""

example3 = """In the current debate over the Future of Work, there is little discussion about how firms anticipate the evolution of their demand for labor and the related mix of skills as they adopt Artificial Intelligence (AI) tools. This article contributes to this debate by leveraging a global survey of 3000 firms in 10 countries, covering the main sectors of the economy. Descriptive statistics from the survey are complemented by econometric analyses of corporate labor demand decisions. The findings are four-fold. First, those are still early days in the absorption of AI technologies, with less than 10% of companies investing in a majority of AI technologies and for multiple purposes. Second, if an aggregate portion of firms anticipates reducing employment as a result of adopting AI technologies, as many other companies anticipate labor growth or reorganizing employment. Third, this reallocation picture holds true when we examine further demand by labor functions and skills, with talent shifting toward more analytic, creative, and interaction skills, and away from administrative and routine-based functions, in line with past trends of skill- and routine-biased technological change. Fourth, a novel to the literature on Future of Work, econometric results on employment change highlight that employment dynamics are driven by related spillover effects to product markets. Higher competition, larger expectations of market (share) deployment may counterbalance negative automation effect on employment dynamics."""
output3 = """This paper examines how firms anticipate changes in labor demand and skill requirements due to AI adoption. It uses a global survey and econometric analyses to assess employment dynamics and skill shifts. The focus on labor market outcomes and the implications of AI adoption qualifies it for category A. Class: A"""

example4 = """This paper develops a set of innovative tools for labor market intelligence by applying machine learning techniques to web vacancies on the Italian labor market. Our approach allows to calculate, for each occupation, the different types of skills required by the market alongside a set of relevant variables such as region, sector, education and level of experience. We construct a taxonomy for skills and map it into the recently developed ESCO classification system. We subsequently develop measures of the relevance of soft and hard skills and we analyze their detailed composition. We apply the dataset constructed to the debate on computerization of work. We show that soft and digital skills are related to the probability of automation of a given occupation and we shed some light on the complementarity/substitutability of hard and soft skills."""
output4 = """This paper applies machine learning to analyze job vacancies in the Italian labor market, focusing on the skills required for different occupations. It examines the relationship between skills and the probability of automation. However, it does not focus on the impact of AI specifically, only computerisation in general. Therefore it is in category B. Class: B"""


In [ ]:
## PROMPT GPT-4o mini again

from openai import OpenAI
from dotenv import load_dotenv
import time
import logging
from tqdm import tqdm
import re

# set up logging
logging.basicConfig(level = logging.INFO)

# Load progress from the last run (if any)
try:
    classed_As = pd.read_csv("classed_As.csv")
    processed_DOIs = set(classed_As['DOI'])
    data_dict = classed_As.to_dict('list')
except FileNotFoundError:
    classed_As = pd.DataFrame(columns=['Abstract', 'DOI', 'GPT_response', 'GPT_class', 'Input_tokens', 'Output_tokens', 'reason_stop', 'system_fingerprint'])
    processed_DOIs = set()
    data_dict = {
        'Abstract': [],
        'DOI': [],
        'GPT_response': [],
        'GPT_class': [],
        'Input_tokens': [],
        'Output_tokens': [],
        'reason_stop': [],
        'system_fingerprint': []
    }

# set up api
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()
client.api_key = api_key

# loop through abstracts
for index, row in tqdm(As.iterrows(), desc = 'Processing abstracts'):
    if row['DOI'] in processed_DOIs:
        continue
        
    else:
        if pd.isnull(row['Abstract']):
            print('blank!')
            data_dict['Abstract'].append(row['Abstract'])
            data_dict['DOI'].append(row['DOI'])
            data_dict['GPT_response'].append('')
            data_dict['GPT_class'].append('')
            data_dict['Input_tokens'].append('')
            data_dict['Output_tokens'].append('')
            data_dict['reason_stop'].append('')
            data_dict['system_fingerprint'].append('')
        else:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                max_tokens=300,
                temperature=0,
                seed=2345,
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": example1},
                    {"role": "assistant", "content": output1},
                    {"role": "user", "content": example2},
                    {"role": "assistant", "content": output2},
                    {"role": "user", "content": example3}, 
                    {"role": "assistant", "content": output3},
                    {"role": "user", "content": example4},
                    {"role": "assistant", "content": output4},
                    {"role": "user", "content": example5},
                    {"role": "assistant", "content": output5},
                    {"role": "user", "content": example6},
                    {"role": "assistant", "content": output6},
                    {"role": "user", "content": example7},
                    {"role": "assistant", "content": output7},
                    {"role": "user", "content": example8},
                    {"role": "assistant", "content": output8},
                    {"role": "user", "content": example9},
                    {"role": "assistant", "content": output9},
                    {"role": "user", "content": example10},
                    {"role": "assistant", "content": output10},
                    {"role": "user", "content": example11},
                    {"role": "assistant", "content": output11},
                    {"role": "user", "content": example12},
                    {"role": "assistant", "content": output12},
                    {"role": "user", "content": example13},
                    {"role": "assistant", "content": output13},
                    {"role": "user", "content": example14},
                    {"role": "assistant", "content": output14},
                    {"role": "user", "content": row['Abstract']}
                ]
            )
        
            # store data in df
            data_dict['Abstract'].append(row['Abstract'])
            data_dict['DOI'].append(row['DOI'])
            message = response.choices[0].message.content
            data_dict['GPT_response'].append(message)
            try:
                data_dict['GPT_class'].append(re.search(r'Class:\s*([A-Z])', message).group(1))
            except ValueError:
                data_dict['GPT_class'].append('')
            data_dict['Input_tokens'].append(response.usage.prompt_tokens)
            data_dict['Output_tokens'].append(response.usage.completion_tokens)
            data_dict['reason_stop'].append(response.choices[0].finish_reason)
            data_dict['system_fingerprint'].append(response.system_fingerprint)
    
        # create df out of dict
        classed_As = pd.DataFrame(data_dict)
        
        # save progress, add url to processed url list
        if (index + 1) % 10 == 0:
            try:
                if len(classed_As) - len(pd.read_csv("classed_As.csv")) == 10:
                    new_papers = classed_As.tail(10)
                    new_papers.to_csv("classed_As.csv", index=False, mode='a', header=False)
                    logging.info("Saved progress at iteration {}".format(index + 1))
                else:
                    logging.info("Warning: trying to overwrite with blank at iteration {}".format(index + 1))
                    break
            except FileNotFoundError:
                classed_As.to_csv("classed_As.csv", index=False, header = True)
        
        # add to list of processed urls
        processed_DOIs.add(row['DOI'])

    # # sleep to observe rate limit (500 requests per minute)
    # time.sleep(0.01)

# Display the DataFrame
classed_As = pd.DataFrame(data_dict)
classed_As.to_csv("classed_As.csv", index=False)
classed_As

Topic 10: As as GPTS, innovation and R&D

In [5]:
example1 = """The recent breakthrough of a net energy gain from a fusion reaction renders tangible several avenues for fusion energy. This article argues that fusion has the potential to be the next General-Purpose Technology (GPT) due to its several non-electricity applications. Following the evolutionary theory of technological change, this paper analyzes the evolution of fusion technologies at two different levels. At the micro level, this paper analyzes a possible evolution path for the fusion industry and for other industries that will be impacted by the developments of fusion, such as transportation and aerospace. At the macro level, this paper analyzes a possible evolution path for the diffusion of fusion within each national innovation system and across countries, a global commission. Both levels of analysis share the need for managed co-opetition via open science, where competitors cooperate under the supervision of national or international agencies. This article concludes with several perspectives that provide insightful contributions for policymakers, practitioners, and politicians interested in understanding how the evolution of fusion technologies will likely affect the funding of research and development and the direction and progress of technological change, and it illustrates the dynamic relation between a GPT and economic growth."""
output1 = """This paper discusses fusion energy as a potential General-Purpose Technology (GPT) and its implications for various industries and economic growth. Since it does not examine AI or its impact on economic outcomes, it falls under Class B. Class: B"""

example2 = """This paper constructs an endogenous growth model that identifies three patterns in the division of labor in terms of innovation between GPT and SPT sectors: (1) the SPT stage, (2) the GPT-SPT joint-research stage, and (3) the autonomous GPT stage. It is shown that the emergence of GPT only has a temporary level effect, and a negative effect on economic growth. However, the new phenomenon of the autonomous GPT stage has a positive influence on both growth and level effects. This result theoretically explains the emergence and resolution of the IT productivity paradox."""
output2 = """This paper constructs an endogenous growth model to analyze the relationship between General-Purpose Technology (GPT) and Specific-Purpose Technology (SPT) sectors. It explores how the different stages of GPT development impact economic growth, providing theoretical insights into the IT productivity paradox. Since this paper does not examine AI or its impacts on economic outcomes (just GPTs in general), it should be classified as Class B. Class: B"""

example3 = """Clusters, which are examples of Porter's diamond in action, may foster creativity. Regional economic development, innovation efforts, and clusters are interdependent. Artificial Intelligence and Big Data can perform activities at a higher level than humans. Recent studies indicate that they are gaining popularity because they can be used to collect, organize, and analyze vast, varied data sets in order to reveal hidden patterns and trends that may be applied to a number of problems. How can Stockholm's Artificial Intelligence and Big Data cluster enhance its competitiveness in light of these factors? How can it be made more appealing? To address these research issues, qualitative research was undertaken using the case study approach and statistical data mostly gathered from the Global Competitiveness Report. Among the most important aspects of this article are the analyses of competitiveness, cluster performance, and main actors of the cluster. The study indicates that Stockholm, the capital of Sweden, has the potential to become one of the world's largest proponents of Artificial Intelligence and Big Data, since it is far more developed than other cities. In addition, a strategy to strengthen the cluster's competitiveness by stressing the implemented actions is proposed."""
output3 = """This paper examines the role of AI and Big Data in enhancing the competitiveness of Stockholm's innovation cluster. It employs qualitative research and case studies to analyze cluster performance and competitiveness. The focus on AI's implications for regional economic development and innovation qualifies it for category A. Class: A"""

example4 = """Collaborating with one competitor is difficult but collaborating with several competitors is a monumental challenge. However, multi-competitor coopetition, or cooperation between multiple competitors, is increasing. This study examines how recent advancements in artificial intelligence (AI) and blockchain can support multi-competitor coopetition by enhancing governance. Examining two coopetitive R&D consortia in pharmaceuticals and medical imaging, we find that a nascent form of AI called federated learning can address key coopetition concerns such proprietary and confidential data protection, knowledge leakage, data sovereignty and silos thereby maintaining organisational boundaries and autonomy. The use of federated learning and blockchain increases transparency and accountability, which reduces information asymmetries and power differential inequities. Together, these technologies decentralise governance and authority, reducing the tension between collective value creation and individual value appropriation inherent in coopetition, particularly those with multiple competitors. Finally, this study illustrates how emerging technologies challenge traditional assumptions about organisational boundaries, distributed innovation, and coopetition."""
output4 = """This paper examines how AI and blockchain can facilitate multi-competitor cooperation (coopetition) in R&D. It focuses on governance improvements and the implications of these technologies for organizational boundaries and innovation. The study highlights the economic and management outcomes of AI in a specific context. Class: A"""

In [ ]:
## PROMPT GPT-4o mini again

from openai import OpenAI
from dotenv import load_dotenv
import time
import logging
from tqdm import tqdm
import re

# set up logging
logging.basicConfig(level = logging.INFO)

# Load progress from the last run (if any)
try:
    classed_As = pd.read_csv("classed_As.csv")
    processed_DOIs = set(classed_As['DOI'])
    data_dict = classed_As.to_dict('list')
except FileNotFoundError:
    classed_As = pd.DataFrame(columns=['Abstract', 'DOI', 'GPT_response', 'GPT_class', 'Input_tokens', 'Output_tokens', 'reason_stop', 'system_fingerprint'])
    processed_DOIs = set()
    data_dict = {
        'Abstract': [],
        'DOI': [],
        'GPT_response': [],
        'GPT_class': [],
        'Input_tokens': [],
        'Output_tokens': [],
        'reason_stop': [],
        'system_fingerprint': []
    }

# set up api
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()
client.api_key = api_key

# loop through abstracts
for index, row in tqdm(As.iterrows(), desc = 'Processing abstracts'):
    if row['DOI'] in processed_DOIs:
        continue
        
    else:
        if pd.isnull(row['Abstract']):
            print('blank!')
            data_dict['Abstract'].append(row['Abstract'])
            data_dict['DOI'].append(row['DOI'])
            data_dict['GPT_response'].append('')
            data_dict['GPT_class'].append('')
            data_dict['Input_tokens'].append('')
            data_dict['Output_tokens'].append('')
            data_dict['reason_stop'].append('')
            data_dict['system_fingerprint'].append('')
        else:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                max_tokens=300,
                temperature=0,
                seed=2345,
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": example1},
                    {"role": "assistant", "content": output1},
                    {"role": "user", "content": example2},
                    {"role": "assistant", "content": output2},
                    {"role": "user", "content": example3}, 
                    {"role": "assistant", "content": output3},
                    {"role": "user", "content": example4},
                    {"role": "assistant", "content": output4},
                    {"role": "user", "content": example5},
                    {"role": "assistant", "content": output5},
                    {"role": "user", "content": example6},
                    {"role": "assistant", "content": output6},
                    {"role": "user", "content": example7},
                    {"role": "assistant", "content": output7},
                    {"role": "user", "content": example8},
                    {"role": "assistant", "content": output8},
                    {"role": "user", "content": example9},
                    {"role": "assistant", "content": output9},
                    {"role": "user", "content": example10},
                    {"role": "assistant", "content": output10},
                    {"role": "user", "content": example11},
                    {"role": "assistant", "content": output11},
                    {"role": "user", "content": example12},
                    {"role": "assistant", "content": output12},
                    {"role": "user", "content": example13},
                    {"role": "assistant", "content": output13},
                    {"role": "user", "content": example14},
                    {"role": "assistant", "content": output14},
                    {"role": "user", "content": row['Abstract']}
                ]
            )
        
            # store data in df
            data_dict['Abstract'].append(row['Abstract'])
            data_dict['DOI'].append(row['DOI'])
            message = response.choices[0].message.content
            data_dict['GPT_response'].append(message)
            try:
                data_dict['GPT_class'].append(re.search(r'Class:\s*([A-Z])', message).group(1))
            except ValueError:
                data_dict['GPT_class'].append('')
            data_dict['Input_tokens'].append(response.usage.prompt_tokens)
            data_dict['Output_tokens'].append(response.usage.completion_tokens)
            data_dict['reason_stop'].append(response.choices[0].finish_reason)
            data_dict['system_fingerprint'].append(response.system_fingerprint)
    
        # create df out of dict
        classed_As = pd.DataFrame(data_dict)
        
        # save progress, add url to processed url list
        if (index + 1) % 10 == 0:
            try:
                if len(classed_As) - len(pd.read_csv("classed_As.csv")) == 10:
                    new_papers = classed_As.tail(10)
                    new_papers.to_csv("classed_As.csv", index=False, mode='a', header=False)
                    logging.info("Saved progress at iteration {}".format(index + 1))
                else:
                    logging.info("Warning: trying to overwrite with blank at iteration {}".format(index + 1))
                    break
            except FileNotFoundError:
                classed_As.to_csv("classed_As.csv", index=False, header = True)
        
        # add to list of processed urls
        processed_DOIs.add(row['DOI'])

    # # sleep to observe rate limit (500 requests per minute)
    # time.sleep(0.01)

# Display the DataFrame
classed_As = pd.DataFrame(data_dict)
classed_As.to_csv("classed_As.csv", index=False)
classed_As

Topic 11: Banking, Credit, Bankrupcy

In [ ]:
example1 = """Historical data of the box office of Russian cinema is the object of research. The purpose of the study is to determine the possibility of forecasting the cash fees of the film project at an early stage in the production of films, which is especially important due to withdrawal of foreign distributors from the Russian market. The analysis was carried out on data for the entire population (N = 1400) of Russian national films that were released from the beginning of 2004 to April 2022. These data are introduced into scientific circulation for the first time. The study used methods of evaluation of film projects based on historical profitability and classification of films by genres, directors, screenwriters. The result of the experiment on 7 machine learning and neural network models achieved an accuracy of 0.96 and ROC (AUC) = 0.98. The article provides conclusions about the directions for improving forecasting methods and conclusions about the limitations of the proposed approach. Taking into account the high volatility of the individual financial result of a film project, recommendations were made by the “portfolio” principle of investment, which opens the prospects of debt and equity financing of cinema using market financial instruments, issuance of bonds and shares by producers and distributors."""
output1 = """This paper focuses on forecasting box office revenues for Russian films using historical data and machine learning models. It discusses the financial implications of these forecasts for investment strategies in the film industry. While it employs machine learning techniques, the primary focus is on economic outcomes related to film financing rather than the impact of AI itself, so it is not relevant. Class: B."""

example2 = """Research background: In the past, the main objective of a company was to generate sufficient profit. Nowadays, a company must seek to achieve much broader objectives. To be successful in this pursuit, it must not only measure financial performance, but also monitor internal and external developments, increase shareholders' wealth and protect the interests of other stakeholders, i.e. to analyze and act on those factors that affect company value. Purpose of the article: The objective of the contribution is to determine through the use of artificial neural networks the relationship between business value drivers, or value based drivers (VBD), and EVA Equity, which is economic value added (EVA), of small and medium-sized enterprises operating in the rural areas of the Czech Republic. Methods: The data was obtained from the Bisnode's Albertina database. The data set consists of the profit and loss accounts for 2013 to 2017 of small and medium-sized enterprises operating in rural areas of the Czech Republic. Two scenarios are analyzed. In the first, the independent variables are only the value drivers, whereas in the second, company location (region) is included. The objective is to find the dependence of EVA Equity on individual VBD and company location. A sensitivity analysis is conducted, on the basis of which the importance of individual value drivers and company location is determined. Findings & Value added: The output is a set of value drivers, which could be used by company managers to regulate the growth of EVA Equity, i.e. value for shareholders. The findings reveal that the difference between successful and unsuccessful companies is determined by the level of involvement of human capital; companies use a large number of substitutes for factors of production, whereby the involvement of borrowed capital is likely to cause a positive financial leverage effect."""
output2 = """This paper investigates the relationship between business value drivers and economic value added (EVA) using artificial neural networks. It focuses on small and medium-sized enterprises in rural Czech Republic and analyzes how these drivers affect EVA. However, it only uses neural networks as a method, it does not investigate their economic implications - therefore it is not relevant. Class: B"""

example3 = """Profit-driven artificial intelligence (AI) systems and profit-based performance measures are widely used in credit scoring. When assessing the performance of an AI system for credit scoring, previous research typically assumes that the cost and benefit parameters and their distributional information are available. In reality, however, these parameters and their distributions are often not precisely known. This study considers parameter uncertainty in the development of credit-scoring models and the estimation of profits and risks generated by those models. We propose a novel profit-based metric—the worst-case expected minimum cost (WEMC)—to estimate the profit of credit-scoring models with uncertain parameters. Furthermore, we introduce the worst-case conditional value-at-risk (WCVaR) metric to measure the loss incurred from employing a classification model in credit scoring under the deterioration of cost parameters. A multiobjective feature-selection framework based on WEMC (or minimum cost) and WCVaR is then presented for model development. Using a comprehensive bankruptcy database, we compare the proposed methods with wrapper methods that use traditional metrics as selection criteria, as well as filter and embedding methods. We conduct extensive experiments to evaluate the economic benefits of the proposed methods under different scenarios that simulate dynamic changes in macroeconomic conditions. The results suggest that the proposed methods outperform other feature-selection methods in the aspects of profit and risk performance metrics in most cases."""
output3 = """This paper focuses on developing new metrics for evaluating AI systems in credit scoring, specifically addressing parameter uncertainty.Because the study focuses on developing a new AI technique, rather than the implications of AI adoption itself, it is not relevant. Class: A."""

example4 = """AI and ML are increasingly being used in risk assessment for lending for assessing the credit worthiness of individuals and companies. The use of AI and ML in credit risk assessment is poised to revolutionize the finance sector and provide banks with better insights into customers' financial behavior and risk profiles. AI and ML methods can automate tedious risk assessment tasks, uncover hidden patterns, develop powerful predictive models, and enable banks to accurately assess the creditworthiness of borrowers. This paper covers different aspects of employing AI and ML for credit risk assessment, discusses current trends, and suggests potential future directions. We conclude that the adoption of AI and ML in banking is gaining momentum and is likely to become the de facto solution for credit risk assessment in the years to come."""
output4 = """This paper discusses the use of AI and ML in credit risk assessment within the finance sector. It focuses on the impact of the adoption of AI and ML in this sector, which qualifies it for category A. Class: A"""

In [ ]:
## PROMPT GPT-4o mini again

from openai import OpenAI
from dotenv import load_dotenv
import time
import logging
from tqdm import tqdm
import re

# set up logging
logging.basicConfig(level = logging.INFO)

# Load progress from the last run (if any)
try:
    classed_As = pd.read_csv("classed_As.csv")
    processed_DOIs = set(classed_As['DOI'])
    data_dict = classed_As.to_dict('list')
except FileNotFoundError:
    classed_As = pd.DataFrame(columns=['Abstract', 'DOI', 'GPT_response', 'GPT_class', 'Input_tokens', 'Output_tokens', 'reason_stop', 'system_fingerprint'])
    processed_DOIs = set()
    data_dict = {
        'Abstract': [],
        'DOI': [],
        'GPT_response': [],
        'GPT_class': [],
        'Input_tokens': [],
        'Output_tokens': [],
        'reason_stop': [],
        'system_fingerprint': []
    }

# set up api
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()
client.api_key = api_key

# loop through abstracts
for index, row in tqdm(As.iterrows(), desc = 'Processing abstracts'):
    if row['DOI'] in processed_DOIs:
        continue
        
    else:
        if pd.isnull(row['Abstract']):
            print('blank!')
            data_dict['Abstract'].append(row['Abstract'])
            data_dict['DOI'].append(row['DOI'])
            data_dict['GPT_response'].append('')
            data_dict['GPT_class'].append('')
            data_dict['Input_tokens'].append('')
            data_dict['Output_tokens'].append('')
            data_dict['reason_stop'].append('')
            data_dict['system_fingerprint'].append('')
        else:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                max_tokens=300,
                temperature=0,
                seed=2345,
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": example1},
                    {"role": "assistant", "content": output1},
                    {"role": "user", "content": example2},
                    {"role": "assistant", "content": output2},
                    {"role": "user", "content": example3}, 
                    {"role": "assistant", "content": output3},
                    {"role": "user", "content": example4},
                    {"role": "assistant", "content": output4},
                    {"role": "user", "content": example5},
                    {"role": "assistant", "content": output5},
                    {"role": "user", "content": example6},
                    {"role": "assistant", "content": output6},
                    {"role": "user", "content": example7},
                    {"role": "assistant", "content": output7},
                    {"role": "user", "content": example8},
                    {"role": "assistant", "content": output8},
                    {"role": "user", "content": example9},
                    {"role": "assistant", "content": output9},
                    {"role": "user", "content": example10},
                    {"role": "assistant", "content": output10},
                    {"role": "user", "content": example11},
                    {"role": "assistant", "content": output11},
                    {"role": "user", "content": example12},
                    {"role": "assistant", "content": output12},
                    {"role": "user", "content": example13},
                    {"role": "assistant", "content": output13},
                    {"role": "user", "content": example14},
                    {"role": "assistant", "content": output14},
                    {"role": "user", "content": row['Abstract']}
                ]
            )
        
            # store data in df
            data_dict['Abstract'].append(row['Abstract'])
            data_dict['DOI'].append(row['DOI'])
            message = response.choices[0].message.content
            data_dict['GPT_response'].append(message)
            try:
                data_dict['GPT_class'].append(re.search(r'Class:\s*([A-Z])', message).group(1))
            except ValueError:
                data_dict['GPT_class'].append('')
            data_dict['Input_tokens'].append(response.usage.prompt_tokens)
            data_dict['Output_tokens'].append(response.usage.completion_tokens)
            data_dict['reason_stop'].append(response.choices[0].finish_reason)
            data_dict['system_fingerprint'].append(response.system_fingerprint)
    
        # create df out of dict
        classed_As = pd.DataFrame(data_dict)
        
        # save progress, add url to processed url list
        if (index + 1) % 10 == 0:
            try:
                if len(classed_As) - len(pd.read_csv("classed_As.csv")) == 10:
                    new_papers = classed_As.tail(10)
                    new_papers.to_csv("classed_As.csv", index=False, mode='a', header=False)
                    logging.info("Saved progress at iteration {}".format(index + 1))
                else:
                    logging.info("Warning: trying to overwrite with blank at iteration {}".format(index + 1))
                    break
            except FileNotFoundError:
                classed_As.to_csv("classed_As.csv", index=False, header = True)
        
        # add to list of processed urls
        processed_DOIs.add(row['DOI'])

    # # sleep to observe rate limit (500 requests per minute)
    # time.sleep(0.01)

# Display the DataFrame
classed_As = pd.DataFrame(data_dict)
classed_As.to_csv("classed_As.csv", index=False)
classed_As

Topic 12: Audit and Accounting

In [ ]:
example1 = """In this paper, we survey public accounting professionals to gauge the extent to which Artificial Intelligence (AI), specifically Robotic Process Automation (RPA) and Machine Learning (ML), are currently being utilized, as well as perceptions about the impact and receptiveness to this technology. Quantitative and qualitative responses from 90 participants, representing various firms, service lines, and positions, indicate that both RPA and ML are currently not being used extensively by public accountants nor by their clients, and firms are conducting some, but not extensive training on these technologies. However, respondents strongly indicated that AI will significantly impact their daily responsibilities in five years and public accountants are very receptive to these changes. Additionally, we find that firm size appears to be the most significant factor impacting differences in responses. These results indicate that while large-scale AI adoption has not yet come to public accounting, substantial changes are on the horizon."""
output1 = """This paper surveys public accounting professionals to assess the current use and perceptions of AI technologies like RPA and ML. It focuses on the adoption of AI in the accounting sector and its anticipated impact on daily responsibilities, which relates to management outcomes, and which qualifies it for category A. Class: A"""

example2 = """Purpose: The nature and amount of data that public organizations have to monitor to counteract corruption lead to a phenomenon called “KPI overload”, consisting of the business analyst feeling overwhelmed by the amount of information and resulting in the absence of appropriate control. The purpose of this study is to develop a solution based on Artificial Intelligence technology to avoid data overloading and, at the same time, under-controlling in business process monitoring. Design/methodology/approach: The authors adopted a design science research approach. The authors started by observing a specific problem in a real context (a healthcare organization); then conceptualized, designed and implemented a solution to the problem with the goal to develop knowledge that can be used to design solutions for similar problems. The proposed solution for business process monitoring integrates databases and self-service business intelligence for outlier detection and artificial intelligence for classification analysis. Findings: The authors found the solution powerful to solve problems related to KPI overload in process monitoring. In the specific case study, the authors found that the combination of Business Intelligence and Artificial Intelligence can provide a significant contribution to the detection of fraud, corruption and/or policy misalignment in public organizations. Originality/value: The authors provide a big-data-based solution to the problem of data overload in business process monitoring that does not sacrifice any monitored Key Performance Indicators and that also reduces the workload of the business analyst. The authors also developed and implemented this automated solution in a context where data sensitivity and privacy are critical issues."""
output2 = """This paper addresses the issue of KPI overload in public organizations and proposes an AI-based solution for business process monitoring. It employs a design science research approach to develop a system that integrates AI and business intelligence for outlier detection and classification. While it discusses the broader implications of AI on productivity via reduced KPI overload, its focus is on developing a tool rather than these implications themselves, therefore it is not relevant and is in category B. Class: B"""

example3 = """Financial reconciliation (cross-checking independent sources of data) is a time-honored and widespread function in finance and audit. Its objectives are to ensure completeness, timeliness, and accuracy of recording of transactions. With the proliferation of data over recent decades, the ways in which reconciliation is approached have evolved. There are now a number of software products that promise to automate the process of detailed transaction matching. However, the 'state of the art' in these products is rules-based reconciliation. Rules-based systems in any domain tend to be brittle in that any change in the data streams causes the rules to have to be debugged and rebuilt, itself often a time-consuming process. And to make matters worse, this might be caused not just by data schema changes, but the contents of what is in the data fields themselves. Either of these can occur if a third party such as a bank changes its internal processes. In a sense, automated reconciliation is where machine translation was almost 70 years ago; IBM's 1954 Georgetown experiment approached Russian-to-English translation using rules, but it took another 4 decades for researchers in data-driven Artificial Intelligence (AI) to realize why machine translation did not initially live up to its promises and develop a truly robust methodology for machine translation based on unsupervised learning. It turns out that financial reconciliation can be cast as a machine translation problem based on unsupervised learning. To our knowledge, we are the first to propose this. Here, we demonstrate via experiments on real-life (albeit small-scale) financial data that this way of approaching the problem demonstrates promise in terms of accuracy, as well as solving the problem of lack of robustness inherent in rules-based approaches."""
output3 = """This paper proposes a novel approach to financial reconciliation by framing it as a machine translation problem using unsupervised learning. It discusses the limitations of current rules-based systems and demonstrates the potential of AI to improve accuracy and robustness in reconciliation processes. While it discusses a specific application of AI, it focuses on developing a method and not analysing the impacts or implications. Class: B"""

example4 = """Artificial intelligence (AI) systems have significantly changed the audit process; nevertheless, the AI revolution opponents view this growth as a step-back as many auditors will fail to adapt to this new environment and will drop behind. Our descriptive study examines the perceived ease of use, usefulness, and contribution to the audit quality of different AI’s types. To address local audit firms concerns about their readiness to use AI systems in auditing processes and to advance auditing research, we examine whether perceived ease of use, usefulness, and contribution to audit quality vary by AI systems type (Assisted, Augmented, and Autonomous). An online questionnaire was used to collect data from 124 auditors representing local audit firms in Jordan. Our results indicate that auditors perceive Assisted and Augmented AI systems as ease of use in auditing while perceiving Autonomous AI systems as complicated to use. Besides, Auditors are underestimating Autonomous AI systems’ capabilities and perceived it as not useful for auditing. The results also reveal a significant difference between the perceived contribution to audit quality by the three AI systems types. This study contributes to the existing literature on AI and auditing by developing and testing a measure for AI systems’ perceived contribution to audit quality. This study also provides empirical evidence on how Jordan local firms auditors perceive AI use in auditing."""
output4 = """This paper examines the perceived ease of use and usefulness of different types of AI systems in the auditing process. It uses an online questionnaire to gather data from auditors in Jordan, focusing on how these perceptions affect audit quality. The study contributes to understanding AI adoption in auditing, which qualifies it for category A. Class: A"""

In [ ]:
## PROMPT GPT-4o mini again

from openai import OpenAI
from dotenv import load_dotenv
import time
import logging
from tqdm import tqdm
import re

# set up logging
logging.basicConfig(level = logging.INFO)

# Load progress from the last run (if any)
try:
    classed_As = pd.read_csv("classed_As.csv")
    processed_DOIs = set(classed_As['DOI'])
    data_dict = classed_As.to_dict('list')
except FileNotFoundError:
    classed_As = pd.DataFrame(columns=['Abstract', 'DOI', 'GPT_response', 'GPT_class', 'Input_tokens', 'Output_tokens', 'reason_stop', 'system_fingerprint'])
    processed_DOIs = set()
    data_dict = {
        'Abstract': [],
        'DOI': [],
        'GPT_response': [],
        'GPT_class': [],
        'Input_tokens': [],
        'Output_tokens': [],
        'reason_stop': [],
        'system_fingerprint': []
    }

# set up api
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()
client.api_key = api_key

# loop through abstracts
for index, row in tqdm(As.iterrows(), desc = 'Processing abstracts'):
    if row['DOI'] in processed_DOIs:
        continue
        
    else:
        if pd.isnull(row['Abstract']):
            print('blank!')
            data_dict['Abstract'].append(row['Abstract'])
            data_dict['DOI'].append(row['DOI'])
            data_dict['GPT_response'].append('')
            data_dict['GPT_class'].append('')
            data_dict['Input_tokens'].append('')
            data_dict['Output_tokens'].append('')
            data_dict['reason_stop'].append('')
            data_dict['system_fingerprint'].append('')
        else:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                max_tokens=300,
                temperature=0,
                seed=2345,
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": example1},
                    {"role": "assistant", "content": output1},
                    {"role": "user", "content": example2},
                    {"role": "assistant", "content": output2},
                    {"role": "user", "content": example3}, 
                    {"role": "assistant", "content": output3},
                    {"role": "user", "content": example4},
                    {"role": "assistant", "content": output4},
                    {"role": "user", "content": example5},
                    {"role": "assistant", "content": output5},
                    {"role": "user", "content": example6},
                    {"role": "assistant", "content": output6},
                    {"role": "user", "content": example7},
                    {"role": "assistant", "content": output7},
                    {"role": "user", "content": example8},
                    {"role": "assistant", "content": output8},
                    {"role": "user", "content": example9},
                    {"role": "assistant", "content": output9},
                    {"role": "user", "content": example10},
                    {"role": "assistant", "content": output10},
                    {"role": "user", "content": example11},
                    {"role": "assistant", "content": output11},
                    {"role": "user", "content": example12},
                    {"role": "assistant", "content": output12},
                    {"role": "user", "content": example13},
                    {"role": "assistant", "content": output13},
                    {"role": "user", "content": example14},
                    {"role": "assistant", "content": output14},
                    {"role": "user", "content": row['Abstract']}
                ]
            )
        
            # store data in df
            data_dict['Abstract'].append(row['Abstract'])
            data_dict['DOI'].append(row['DOI'])
            message = response.choices[0].message.content
            data_dict['GPT_response'].append(message)
            try:
                data_dict['GPT_class'].append(re.search(r'Class:\s*([A-Z])', message).group(1))
            except ValueError:
                data_dict['GPT_class'].append('')
            data_dict['Input_tokens'].append(response.usage.prompt_tokens)
            data_dict['Output_tokens'].append(response.usage.completion_tokens)
            data_dict['reason_stop'].append(response.choices[0].finish_reason)
            data_dict['system_fingerprint'].append(response.system_fingerprint)
    
        # create df out of dict
        classed_As = pd.DataFrame(data_dict)
        
        # save progress, add url to processed url list
        if (index + 1) % 10 == 0:
            try:
                if len(classed_As) - len(pd.read_csv("classed_As.csv")) == 10:
                    new_papers = classed_As.tail(10)
                    new_papers.to_csv("classed_As.csv", index=False, mode='a', header=False)
                    logging.info("Saved progress at iteration {}".format(index + 1))
                else:
                    logging.info("Warning: trying to overwrite with blank at iteration {}".format(index + 1))
                    break
            except FileNotFoundError:
                classed_As.to_csv("classed_As.csv", index=False, header = True)
        
        # add to list of processed urls
        processed_DOIs.add(row['DOI'])

    # # sleep to observe rate limit (500 requests per minute)
    # time.sleep(0.01)

# Display the DataFrame
classed_As = pd.DataFrame(data_dict)
classed_As.to_csv("classed_As.csv", index=False)
classed_As

Topic 14: Agriculture

In [ ]:
example1 = """This work presents and implements a low-cost irrigation system for smart agriculture that is based on the Internet of Things (IoT). In order to continuously monitor environmental data in real time, the system is equipped with a network of sensors, including pressure, temperature, moisture, and water level sensors. In order to anticipate when irrigation pumps will switch on, machine learning techniques including Artificial Neural Networks (ANN), Decision Trees (DT), Naive Bayes (NB), and Support Vector Machines (SVM) are linked depending on established parameters. The study shows that the ANN model can identify complicated patterns in the agricultural environment with an accuracy of up to 98.33%. Farmers are able to make well-informed decisions quickly thanks to the cloud connectivity and intuitive interface of remote monitoring and control. Because predictive modeling minimizes pump activation delays, it lessens the chance of both over- and under-irrigation. The suggested strategy makes the most use of available water and provides opportunities for precision farming, which is a major step forward for sustainable agriculture. The study's findings demonstrate how well the system uses resources and open the door for the future creation of innovative, scalable agricultural technology."""
output1 = """This paper develops a low-cost irrigation system for smart agriculture, using IoT and machine learning as tools to examine and improve irrigation practices. Since AI is not the central focus but rather a means to achieve operational improvements, it fits into Class B."""

example2 = """The agri-food sector contributes significantly to economic and social advancements globally despite numerous challenges such as food safety and security, demand and supply gaps, product quality, traceability, etc. Digital technologies offer effective and sustainable ways to these challenges through reduced human interference and improved data-accuracy. Innovations led by digital transformations in the agri-food supply chains (AFSCs) are the main aim of 'Agri-Food 4.0'. This brings significant transformations in the agri-food sector by reducing food wastage, real-time product monitoring, reducing scalability issues, etc. This paper presents a systematic review of the innovations in the agri-food for digital technologies such as internet-of-things, artificial intelligence, big data, RFID, robotics, block-chain technology, etc. The employment of these technologies from the 'farm to fork' along AFSC emphasizes a review of 159 articles solicited from different sources. This paper also highlights digitization in developing smart, sensible, and sustainable agri-food supply chain systems."""
output2 = """This paper systematically reviews the innovations in the agri-food sector driven by digital technologies, including AI. It discusses the implications of these technologies for improving supply chains and addressing challenges like food safety and waste. The focus on economic and management outcomes qualifies it for category A. Class: A"""

example3 = """As population aging becomes the new demographic norm in China, its workforce structure is changing, and its demographic dividend is about to disappear. Artificial intelligence (AI) has been experiencing rapid progress in the last few years, and it is becoming an important tool to address the impact and challenges of an aging population. Therefore, this research introduces population aging and artificial intelligence into agricultural production, focusing on the effects of population aging on food security and the function played by artificial intelligence in it. From an empirical study conducted based on provincial panel data, the following conclusions arise. First, population aging in rural China has not negatively impacted food security, so there is no need to be overly pessimistic about the inevitable aging of the rural population. Second, AI has a favorable moderating function on the effects of population aging on food security. Third, the moderating effect of AI is heterogeneous. Compared with other provinces, AI can play a strongly positive moderating effect in central and west regions and major grain-selling areas. Based on the above findings, this paper proposes targeted policy recommendations on protecting food security in the context of artificial intelligence and population aging."""
output3 = """This paper examines the relationship between population aging, AI, and food security in rural China. It uses empirical data to analyze how AI moderates the effects of aging on food security. The focus on AI's role in addressing economic challenges qualifies it for category A. Class: A"""

example4 = """The problem of multiple zones in computer vision, including pattern recognition in the agricultural sector, occupies a special place in the field of artificial intelligence in the modern aspect. The object of the study is the recognition of weeds based on deep learning and computer vision. The subject of the study is the effective use of neural network models in training, involving classification and processing using datasets of plants and weeds. The relevance of the study lies in the demand of the modern world in the use of new information technologies in industrial agriculture, which contributes to improving the efficiency of agro-industrial complexes. The interest of private agricultural enterprises and the state is caused by an increase in the yield of agricultural products. To recognize weeds, machine learning methods, in particular neural networks, were used. The process of weed recognition is described using the Mark model, as a result of processing 1,562 pictures, segmented images are obtained. Due to the annual increase in weeds on the territory of Kazakhstan and in the course of solving these problems, a new plant recognition code was developed and written in the scanner software module. The scanner, in turn, provides automatic detection of weeds. Based on the results of a trained neural network based on the MaskRCNN neural network model written in the scanner software module meeting new time standards, the automated plant scanning and recognition system was improved. The weed was recognized in an average of 0.2 seconds with an accuracy of 89 %, while the additional human factor was completely removed. The use of new technology helps to control weeds and contributes to solving the problem of controlling them."""
output4 = """This study focuses on using deep learning and computer vision for weed recognition in agriculture. The study uses AI and machine learning solely as methods to improve weed recognition, not focusing on the broader economic implications of AI. Thus, it correctly falls into Class B. Class: B"""

In [ ]:
## PROMPT GPT-4o mini again

from openai import OpenAI
from dotenv import load_dotenv
import time
import logging
from tqdm import tqdm
import re

# set up logging
logging.basicConfig(level = logging.INFO)

# Load progress from the last run (if any)
try:
    classed_As = pd.read_csv("classed_As.csv")
    processed_DOIs = set(classed_As['DOI'])
    data_dict = classed_As.to_dict('list')
except FileNotFoundError:
    classed_As = pd.DataFrame(columns=['Abstract', 'DOI', 'GPT_response', 'GPT_class', 'Input_tokens', 'Output_tokens', 'reason_stop', 'system_fingerprint'])
    processed_DOIs = set()
    data_dict = {
        'Abstract': [],
        'DOI': [],
        'GPT_response': [],
        'GPT_class': [],
        'Input_tokens': [],
        'Output_tokens': [],
        'reason_stop': [],
        'system_fingerprint': []
    }

# set up api
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()
client.api_key = api_key

# loop through abstracts
for index, row in tqdm(As.iterrows(), desc = 'Processing abstracts'):
    if row['DOI'] in processed_DOIs:
        continue
        
    else:
        if pd.isnull(row['Abstract']):
            print('blank!')
            data_dict['Abstract'].append(row['Abstract'])
            data_dict['DOI'].append(row['DOI'])
            data_dict['GPT_response'].append('')
            data_dict['GPT_class'].append('')
            data_dict['Input_tokens'].append('')
            data_dict['Output_tokens'].append('')
            data_dict['reason_stop'].append('')
            data_dict['system_fingerprint'].append('')
        else:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                max_tokens=300,
                temperature=0,
                seed=2345,
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": example1},
                    {"role": "assistant", "content": output1},
                    {"role": "user", "content": example2},
                    {"role": "assistant", "content": output2},
                    {"role": "user", "content": example3}, 
                    {"role": "assistant", "content": output3},
                    {"role": "user", "content": example4},
                    {"role": "assistant", "content": output4},
                    {"role": "user", "content": example5},
                    {"role": "assistant", "content": output5},
                    {"role": "user", "content": example6},
                    {"role": "assistant", "content": output6},
                    {"role": "user", "content": example7},
                    {"role": "assistant", "content": output7},
                    {"role": "user", "content": example8},
                    {"role": "assistant", "content": output8},
                    {"role": "user", "content": example9},
                    {"role": "assistant", "content": output9},
                    {"role": "user", "content": example10},
                    {"role": "assistant", "content": output10},
                    {"role": "user", "content": example11},
                    {"role": "assistant", "content": output11},
                    {"role": "user", "content": example12},
                    {"role": "assistant", "content": output12},
                    {"role": "user", "content": example13},
                    {"role": "assistant", "content": output13},
                    {"role": "user", "content": example14},
                    {"role": "assistant", "content": output14},
                    {"role": "user", "content": row['Abstract']}
                ]
            )
        
            # store data in df
            data_dict['Abstract'].append(row['Abstract'])
            data_dict['DOI'].append(row['DOI'])
            message = response.choices[0].message.content
            data_dict['GPT_response'].append(message)
            try:
                data_dict['GPT_class'].append(re.search(r'Class:\s*([A-Z])', message).group(1))
            except ValueError:
                data_dict['GPT_class'].append('')
            data_dict['Input_tokens'].append(response.usage.prompt_tokens)
            data_dict['Output_tokens'].append(response.usage.completion_tokens)
            data_dict['reason_stop'].append(response.choices[0].finish_reason)
            data_dict['system_fingerprint'].append(response.system_fingerprint)
    
        # create df out of dict
        classed_As = pd.DataFrame(data_dict)
        
        # save progress, add url to processed url list
        if (index + 1) % 10 == 0:
            try:
                if len(classed_As) - len(pd.read_csv("classed_As.csv")) == 10:
                    new_papers = classed_As.tail(10)
                    new_papers.to_csv("classed_As.csv", index=False, mode='a', header=False)
                    logging.info("Saved progress at iteration {}".format(index + 1))
                else:
                    logging.info("Warning: trying to overwrite with blank at iteration {}".format(index + 1))
                    break
            except FileNotFoundError:
                classed_As.to_csv("classed_As.csv", index=False, header = True)
        
        # add to list of processed urls
        processed_DOIs.add(row['DOI'])

    # # sleep to observe rate limit (500 requests per minute)
    # time.sleep(0.01)

# Display the DataFrame
classed_As = pd.DataFrame(data_dict)
classed_As.to_csv("classed_As.csv", index=False)
classed_As

Topic 16: AI in decision-making, ethics, algorithmic bias and fairness

In [2]:
example1 = """Although artificial intelligence can contribute to decision-making processes, many industry players lag behind pioneering companies in utilizing artificial intelligence-driven technologies, which is a significant problem. Explainable artificial intelligence can be a viable solution to mitigate this problem. This paper proposes a research model to address how explainable artificial intelligence can impact decision-making processes. Using an experimental design, empirical data is collected to test the research model. This paper is one of the pioneer papers providing empirical evidence about the impact of explainable artificial intelligence on supply chain decision-making processes. We propose a serial mediation path, which includes transparency and agile decision-making. Findings reveal that explainable artificial intelligence enhances transparency, thereby significantly contributing to agile decision-making for improving cyber resilience during supply chain cyberattacks. Moreover, we conduct a post hoc analysis using text analysis to explore the themes present in tweets discussing explainable artificial intelligence in decision support systems. The results indicate a predominantly positive attitude towards explainable artificial intelligence within these systems. Furthermore, the text analysis reveals two main themes that emphasize the importance of transparency, explainability, and interpretability in explainable artificial intelligence. """
output1 = """This paper examines the impact of explainable artificial intelligence (AI) on decision-making processes in supply chain management, highlighting its role in improving transparency and agile decision-making during cyberattacks. It also explores public perceptions of explainable AI through text analysis, focusing on themes like transparency and interpretability. Since the paper discusses how AI impacts decision-making and operational efficiency within an economic context (supply chains), it aligns with Class A. Class: A"""

example2 = """Now that machine learning algorithms lie at the center of many important resource allocation pipelines, computer scientists have been unwittingly cast as partial social planners. Given this state of affairs, important questions follow. How do leading notions of fairness as defined by computer scientists map onto longer-standing notions of social welfare? In this paper, we present a welfare-based analysis of fair classification regimes. Our main findings assess the welfare impact of fairness-constrained empirical risk minimization programs on the individuals and groups who are subject to their outputs. We fully characterize the ranges of Îµ perturbations to a fairness parameter Îµ in a fair Soft Margin SVM problem that yield better, worse, and neutral outcomes in utility for individuals and by extension, groups. Our method of analysis allows for fast and efficient computation of â€œfairness-to-welfareâ€ solution paths, thereby allowing practitioners to easily assess whether and which fair learning procedures result in classification outcomes that make groups better-off. Our analyses show that applying stricter fairness criteria codified as parity constraints can worsen welfare outcomes for both groups. More generally, always preferring â€œmore fairâ€ classifiers does not abide by the Pareto Principle-a fundamental axiom of social choice theory and welfare economics. Recent work in machine learning has rallied around these notions of fairness as critical to ensuring that algorithmic systems do not have disparate negative impact on disadvantaged social groups. By showing that these constraints often fail to translate into improved outcomes for these groups, we cast doubt on their effectiveness as a means to ensure fairness and justice."""
output2 = """This paper examines the relationship between fairness in machine learning and social welfare. It analyzes the welfare impacts of fairness-constrained algorithms, particularly in classification tasks. The focus on the implications of AI for social welfare and fairness aligns it with economic outcomes. Class: A"""

example3 = """Applying machine learning in real-world applications may have various implications on companies, but individuals as well. Besides obtaining lower costs, faster time to decision and higher accuracy of the decision, automation of decisions can lead to unethical and illegal consequences. More specifically, predictions can systematically discriminate against a certain group of people. This comes mainly due to dataset bias. In this paper, we investigate instances oversampling to improve fairness. We tried several strategies and two techniques, namely SMOTE and random oversampling. Besides traditional oversampling techniques, we tried oversampling of instances based on sensitive attributes as well (i.e. gender or race). We demonstrate on real-world datasets (Adult and COMPAS) that oversampling techniques increase fairness, without greater decrease in predictive accuracy. Oversampling improved fairness up to 15% and AUPRC up to 3% with a loss in AUC of 2%."""
output3 = """This paper explores the use of machine learning to addresses the issue of dataset bias, which can lead to unethical or illegal consequences, such as discrimination. It investigates oversampling techniques to improve fairness in predictions without significantly compromising accuracy. Since AI is used as a method to examine fairness and mitigate bias, rather than focusing on these issues in an economic context / how they relate to economic activity, it should be classified as Class B."""

example4 = """Why do biased algorithmic predictions arise, and what interventions can prevent them? We examine this topic with a field experiment about using machine learning to predict human capital. We randomly assign approximately 400 AI engineers to develop software under different experimental conditions to predict standardized test scores of OECD residents. We then assess the resulting predictive algorithms using the realized test performances, and through randomized audit-like manipulations of algorithmic inputs. We also used the diversity of our subject population to measure whether demographically non-traditional engineers were more likely to notice and reduce algorithmic bias, and whether algorithmic prediction errors are correlated within programmer demographic groups. """
output4 = """This paper investigates algorithmic bias in machine learning through a field experiment involving AI engineers predicting standardized test scores. Since the focus is on algorithmic bias in general rather than the ethical implications of AI within an economic context, it should be classified as Class B. Class: B"""

In [ ]:
## PROMPT GPT-4o mini again

from openai import OpenAI
from dotenv import load_dotenv
import time
import logging
from tqdm import tqdm
import re

# set up logging
logging.basicConfig(level = logging.INFO)

# Load progress from the last run (if any)
try:
    classed_As = pd.read_csv("classed_As.csv")
    processed_DOIs = set(classed_As['DOI'])
    data_dict = classed_As.to_dict('list')
except FileNotFoundError:
    classed_As = pd.DataFrame(columns=['Abstract', 'DOI', 'GPT_response', 'GPT_class', 'Input_tokens', 'Output_tokens', 'reason_stop', 'system_fingerprint'])
    processed_DOIs = set()
    data_dict = {
        'Abstract': [],
        'DOI': [],
        'GPT_response': [],
        'GPT_class': [],
        'Input_tokens': [],
        'Output_tokens': [],
        'reason_stop': [],
        'system_fingerprint': []
    }

# set up api
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()
client.api_key = api_key

# loop through abstracts
for index, row in tqdm(As.iterrows(), desc = 'Processing abstracts'):
    if row['DOI'] in processed_DOIs:
        continue
        
    else:
        if pd.isnull(row['Abstract']):
            print('blank!')
            data_dict['Abstract'].append(row['Abstract'])
            data_dict['DOI'].append(row['DOI'])
            data_dict['GPT_response'].append('')
            data_dict['GPT_class'].append('')
            data_dict['Input_tokens'].append('')
            data_dict['Output_tokens'].append('')
            data_dict['reason_stop'].append('')
            data_dict['system_fingerprint'].append('')
        else:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                max_tokens=300,
                temperature=0,
                seed=2345,
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": example1},
                    {"role": "assistant", "content": output1},
                    {"role": "user", "content": example2},
                    {"role": "assistant", "content": output2},
                    {"role": "user", "content": example3}, 
                    {"role": "assistant", "content": output3},
                    {"role": "user", "content": example4},
                    {"role": "assistant", "content": output4},
                    {"role": "user", "content": example5},
                    {"role": "assistant", "content": output5},
                    {"role": "user", "content": example6},
                    {"role": "assistant", "content": output6},
                    {"role": "user", "content": example7},
                    {"role": "assistant", "content": output7},
                    {"role": "user", "content": example8},
                    {"role": "assistant", "content": output8},
                    {"role": "user", "content": example9},
                    {"role": "assistant", "content": output9},
                    {"role": "user", "content": example10},
                    {"role": "assistant", "content": output10},
                    {"role": "user", "content": example11},
                    {"role": "assistant", "content": output11},
                    {"role": "user", "content": example12},
                    {"role": "assistant", "content": output12},
                    {"role": "user", "content": example13},
                    {"role": "assistant", "content": output13},
                    {"role": "user", "content": example14},
                    {"role": "assistant", "content": output14},
                    {"role": "user", "content": row['Abstract']}
                ]
            )
        
            # store data in df
            data_dict['Abstract'].append(row['Abstract'])
            data_dict['DOI'].append(row['DOI'])
            message = response.choices[0].message.content
            data_dict['GPT_response'].append(message)
            try:
                data_dict['GPT_class'].append(re.search(r'Class:\s*([A-Z])', message).group(1))
            except ValueError:
                data_dict['GPT_class'].append('')
            data_dict['Input_tokens'].append(response.usage.prompt_tokens)
            data_dict['Output_tokens'].append(response.usage.completion_tokens)
            data_dict['reason_stop'].append(response.choices[0].finish_reason)
            data_dict['system_fingerprint'].append(response.system_fingerprint)
    
        # create df out of dict
        classed_As = pd.DataFrame(data_dict)
        
        # save progress, add url to processed url list
        if (index + 1) % 10 == 0:
            try:
                if len(classed_As) - len(pd.read_csv("classed_As.csv")) == 10:
                    new_papers = classed_As.tail(10)
                    new_papers.to_csv("classed_As.csv", index=False, mode='a', header=False)
                    logging.info("Saved progress at iteration {}".format(index + 1))
                else:
                    logging.info("Warning: trying to overwrite with blank at iteration {}".format(index + 1))
                    break
            except FileNotFoundError:
                classed_As.to_csv("classed_As.csv", index=False, header = True)
        
        # add to list of processed urls
        processed_DOIs.add(row['DOI'])

    # # sleep to observe rate limit (500 requests per minute)
    # time.sleep(0.01)

# Display the DataFrame
classed_As = pd.DataFrame(data_dict)
classed_As.to_csv("classed_As.csv", index=False)
classed_As

Topic 18: AI and competition and market structures

In [4]:
example1 = """Producing organic milk has been an alternative for conventional farmers who are struggling with low prices. However, organic dairy prices recently collapsed along with conventional prices when several large producers entered the organic market. The current study explores the patterns and determinants of organic dairy premiums. We estimate monthly price premiums for organic whole milk, other fluid milk, yogurt, and eggs over the period 2008â€“2017 using US retail scanner data. The premiums were 56%, 51%, 27%, and 38%, respectively, with annual compound growth rates of 8%, 7%, âˆ’1.42%, and âˆ’2.86%. We identify the determinants of price premiums with a vector autoregression to account for endogeneity and autocorrelation, and a machine-learning algorithmâ€”extreme gradient boostâ€”for data-driven insights. Holding the premiums fixed, growing input costs may induce organic sellers to sell more to compensate for costs. Holding organic sales fixed, organic premiums decrease in conventional share of organic, cost of diesel, electricity, and feed grains in general; but increase in the number of shoppers and wages."""
output1 = """This study uses machine learning to analyze price premiums for organic dairy products and their determinants. Since AI is used as a tool to study economic outcomes in the agricultural sector, rather than focusing on AI's broader economic implications, it fits into Class B. Class: B"""

example2 = """The widespread use of algorithmic technologies makes rules on tacit collusion, which are already controversial in antitrust law, more complicated. These rules have obvious limitations in effectively regulating algorithmic collusion. Although some scholars and practitioners within antitrust circles in the United States, Europe and beyond have taken notice of this problem, they have failed to a large extent to make clear its specific manifestations, root causes, and effective legal solutions. In this article, the authors make a strong argument that it is no longer appropriate to regard algorithms as mere tools of firms, and that the distinct features of machine learning algorithms as super-tools and as legal persons may inevitably bring about two new cracks in antitrust law. This article clarifies the root causes why these rules are inapplicable to a large extent to algorithmic collusion particularly in the case of machine learning algorithms, classifies the new legal cracks, and provides sound legal criteria for the courts and competition authorities to assess the legality of algorithmic collusion much more accurately. More importantly, this article proposes an efficacious solution to revive the market pricing mechanism for the purposes of resolving the two new cracks identified in antitrust law."""
output2 = """This paper discusses the implications of algorithmic technologies on antitrust law, particularly regarding tacit collusion. It identifies limitations in current regulations and proposes legal solutions to address these challenges. The focus is on the legal and regulatory implications of AI. Class: A"""

example3 = """This study examines a duopoly market comprising an online retail platform and a physical store, both of them selling experience-based products to consumers who are unaware of the productsâ€™ fitness. The platform could introduce artificial intelligence (AI) technology into retail operations to address consumersâ€™ fitness uncertainty and then gain market share, while the physical store could exert a service effort to recapture the market, potentially facilitating showrooming behavior. Operational decisions including pricing, AI application, and service effort exertion are investigated in the cases where consumers are allowed and unallowed to return unsuitable products. We first develop a theoretical model of dual-channel retailing to determine the equilibrium operational decisions for the supply chain members, and then examine the interactions between these operational decisions and consumersâ€™ showrooming behaviors. Subsequently, we perform numerical simulations to verify the robustness of the theoretical findings. Results indicate that both AI application and service efforts exertion will strengthen consumersâ€™ showrooming effect, especially when the cost of AI application is relatively low. Moreover, regardless of whether the store implements the service effort or not, the platform prefers to apply AI technology when consumers are allowed to return products. Furthermore, the physical store will always exert a service effort, and with the service effort, the application of AI technology in the retail platform operation could effectively mitigate the impact of consumer return on supply chain members."""
output3 = """This study investigates the impact of AI technology in a duopoly market involving an online retail platform and a physical store. It develops a theoretical model to analyze operational decisions, including AI application and service efforts, and their effects on consumer behavior (showrooming). The focus on AI's role in operational decisions and market dynamics qualifies it for category A. Class: A"""

example4 = """In online channels, products are returned at high rates. Shipping, processing, and refurbishing are so costly that a retailerâ€™s profit is extremely sensitive to return rates. Using a large data set from a European apparel retailer, we observe that return rates for fashion items bought online range from 13% to 96%, with an average of 53%; many items are not profitable. Because fashion seasons are over before sufficient data on return rates are observed, retailers need to anticipate each itemâ€™s return rate prior to launch. We use product images and traditional measures available prelaunch to predict individual item return rates and decide whether to include the item in the retailerâ€™s assortment. We com-plement machine-based prediction with automatically extracted image-based interpretable features. Insights suggest how to select and design fashion items that are less likely to be returned. Our illustrative machine-learning models predict well and provide face-valid interpretations; the focal retailer can improve profit by 8.3% and identify items with features less likely to be returned. We demonstrate that other machine-learning models do almost as well, reinforcing the value of using prelaunch images to manage returns."""
output4 = """This study uses machine learning to predict return rates for fashion items in online retail, based on product images and other prelaunch data, to help retailers manage returns and improve profitability. It does not discuss the economic impacts of AI. Class: B"""

In [ ]:
## PROMPT GPT-4o mini again

from openai import OpenAI
from dotenv import load_dotenv
import time
import logging
from tqdm import tqdm
import re

# set up logging
logging.basicConfig(level = logging.INFO)

# Load progress from the last run (if any)
try:
    classed_As = pd.read_csv("classed_As.csv")
    processed_DOIs = set(classed_As['DOI'])
    data_dict = classed_As.to_dict('list')
except FileNotFoundError:
    classed_As = pd.DataFrame(columns=['Abstract', 'DOI', 'GPT_response', 'GPT_class', 'Input_tokens', 'Output_tokens', 'reason_stop', 'system_fingerprint'])
    processed_DOIs = set()
    data_dict = {
        'Abstract': [],
        'DOI': [],
        'GPT_response': [],
        'GPT_class': [],
        'Input_tokens': [],
        'Output_tokens': [],
        'reason_stop': [],
        'system_fingerprint': []
    }

# set up api
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()
client.api_key = api_key

# loop through abstracts
for index, row in tqdm(As.iterrows(), desc = 'Processing abstracts'):
    if row['DOI'] in processed_DOIs:
        continue
        
    else:
        if pd.isnull(row['Abstract']):
            print('blank!')
            data_dict['Abstract'].append(row['Abstract'])
            data_dict['DOI'].append(row['DOI'])
            data_dict['GPT_response'].append('')
            data_dict['GPT_class'].append('')
            data_dict['Input_tokens'].append('')
            data_dict['Output_tokens'].append('')
            data_dict['reason_stop'].append('')
            data_dict['system_fingerprint'].append('')
        else:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                max_tokens=300,
                temperature=0,
                seed=2345,
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": example1},
                    {"role": "assistant", "content": output1},
                    {"role": "user", "content": example2},
                    {"role": "assistant", "content": output2},
                    {"role": "user", "content": example3}, 
                    {"role": "assistant", "content": output3},
                    {"role": "user", "content": example4},
                    {"role": "assistant", "content": output4},
                    {"role": "user", "content": example5},
                    {"role": "assistant", "content": output5},
                    {"role": "user", "content": example6},
                    {"role": "assistant", "content": output6},
                    {"role": "user", "content": example7},
                    {"role": "assistant", "content": output7},
                    {"role": "user", "content": example8},
                    {"role": "assistant", "content": output8},
                    {"role": "user", "content": example9},
                    {"role": "assistant", "content": output9},
                    {"role": "user", "content": example10},
                    {"role": "assistant", "content": output10},
                    {"role": "user", "content": example11},
                    {"role": "assistant", "content": output11},
                    {"role": "user", "content": example12},
                    {"role": "assistant", "content": output12},
                    {"role": "user", "content": example13},
                    {"role": "assistant", "content": output13},
                    {"role": "user", "content": example14},
                    {"role": "assistant", "content": output14},
                    {"role": "user", "content": row['Abstract']}
                ]
            )
        
            # store data in df
            data_dict['Abstract'].append(row['Abstract'])
            data_dict['DOI'].append(row['DOI'])
            message = response.choices[0].message.content
            data_dict['GPT_response'].append(message)
            try:
                data_dict['GPT_class'].append(re.search(r'Class:\s*([A-Z])', message).group(1))
            except ValueError:
                data_dict['GPT_class'].append('')
            data_dict['Input_tokens'].append(response.usage.prompt_tokens)
            data_dict['Output_tokens'].append(response.usage.completion_tokens)
            data_dict['reason_stop'].append(response.choices[0].finish_reason)
            data_dict['system_fingerprint'].append(response.system_fingerprint)
    
        # create df out of dict
        classed_As = pd.DataFrame(data_dict)
        
        # save progress, add url to processed url list
        if (index + 1) % 10 == 0:
            try:
                if len(classed_As) - len(pd.read_csv("classed_As.csv")) == 10:
                    new_papers = classed_As.tail(10)
                    new_papers.to_csv("classed_As.csv", index=False, mode='a', header=False)
                    logging.info("Saved progress at iteration {}".format(index + 1))
                else:
                    logging.info("Warning: trying to overwrite with blank at iteration {}".format(index + 1))
                    break
            except FileNotFoundError:
                classed_As.to_csv("classed_As.csv", index=False, header = True)
        
        # add to list of processed urls
        processed_DOIs.add(row['DOI'])

    # # sleep to observe rate limit (500 requests per minute)
    # time.sleep(0.01)

# Display the DataFrame
classed_As = pd.DataFrame(data_dict)
classed_As.to_csv("classed_As.csv", index=False)
classed_As

Topic 20: Smart Cities

In [ ]:
example1 = """Accurately forecasting ridesourcing demand is important for effective transportation planning and policy making. With the rise of Artificial Intelligence (AI), researchers have started to utilize machine learning models to forecast travel demand, which, in many cases, can produce higher prediction accuracy than statistical models. However, most existing machine-learning studies used a global model to predict the demand and ignored the influence of spatial heterogeneity (i.e., the spatial variations in the impacts of explanatory variables). Spatial heterogeneity can drive the parameter estimations varying over space; failing to consider the spatial variations may limit the model's prediction performance. To account for spatial heterogeneity, this study proposes a Clustering-aided Ensemble Method (CEM) to forecast the zone-to-zone (census-tract-to-census-tract) travel demand for ridesourcing services. Specifically, we develop an interactive clustering approach (powered by human in-the-loop AI) to split the origin-destination pairs into different clusters and ensemble the cluster-specific machine learning models for prediction. We implement and test the proposed methodology by using the ridesourcing-trip data in Chicago. The results show that, with a more transparent and flexible model structure, the CEM significantly improves the prediction accuracy than the benchmark models (i.e., global machine learning and statistical models directly trained on all observations). This study offers transportation researchers and practitioners a new methodology of travel demand forecasting, especially for new travel modes like ridesourcing and micromobility."""
output1 = """This paper focuses on forecasting ridesourcing demand using a machine learning model that accounts for spatial heterogeneity. It develops a new methodology (Clustering-aided Ensemble Method) to improve prediction accuracy. While the study is relevant to transportation planning and policy, it focuses on developing an AI technique rather than the implications of AI itself, therefore it is in category B. Class: B"""

example2 = """Cities are increasingly adopting advanced technologies to address complex challenges. Applying technologies such as information and communication technology, artificial intelligence, big data analytics, and autonomous systems in cities' design, planning, and management can cause disruptive changes in their social, economic, and environmental composition. Through a systematic literature review, this research develops a conceptual model linking (1) the dominant city labels relating to tech-driven urban development, (2) the characteristics and applications of disruptive technologies, and (3) the current understanding of inclusive urban development. We extend the discussion by identifying and incorporating the motivations behind adopting disruptive technologies and the challenges they present to inclusive development. We find that inclusive development in tech-driven cities can be realised if governments develop suitable adaptive regulatory frameworks for involving technology companies, build policy capacity, and adopt more adaptive models of governance. We also stress the importance of acknowledging the influence of digital literacy and smart citizenship, and exploring other dimensions of inclusivity, for governing disruptive technologies in inclusive smart cities."""
output2 = """This paper examines the impact of advanced technologies, including AI, on urban development. It develops a conceptual model linking technology adoption to social, economic, and environmental changes in cities. The focus on inclusive urban development and the implications for governance and policy suggests a connection to economic outcomes. Class: A"""

example3 = """Information communication technologies (ICTs) for smart cities enhance the quality of life for residents by incorporating various evolving technologies. It utilises interconnected devices and distinct technologies to optimise energy consumption, improve transportation systems, manage waste efficiently, and enhance public safety. It is claimed that research on smart cities is emerging where experts are trying to uncover the influence of smart cities ICTs from various perspectives and domains, worldwide. This study endeavours to investigate the nexus among smart city ICTs such as IoT, big data analytics, AI, cloud computing, wireless communication, smart grids, intelligent transportation system, e-governance, smart building, smart health, cyber security, and smart education with respect to economic development (ED) from the context of China. This study also attempts to uncover the moderating influence of government policies (GPs) between ICTs for smart city and ED. This study confirmed a positive connection of smart city ICTs on ED. Second, the study revealed a positive moderating connection of GP between smart city ICTs and ED. This study provides insights into smart cities and ICTs that show how shifting paradigm toward smart technologies can help to improve government services."""
output3 = """This paper investigates the relationship between smart city ICTs, including AI, and economic development in China. It also examines the moderating role of government policies. The focus on economic development outcomes and the implications of AI in this context qualifies it for category A. Class: A"""

example4 = """This research provides a novel framework to estimate the economic, environmental, and social values of electrifying public transit buses, for cities across the world, based on open-source data. Electric buses are a compelling candidate to replace diesel buses for their environmental and social benefits. However, the state-of-art models to evaluate the value of bus electrification require granular and bespoke data on bus operation that can be difficult to procure. This strict requirement on data and modeling can hinder potential collaborators on bus electrification, such as electric vehicle manufacturers or electricity providers, from selecting the candidate public transit agencies in the initial phase of the market investigation. Our valuation tool uses General Transit Feed Specification, a standard data format used by transit agencies worldwide, to provide high-level guidance on developing a prioritization strategy for electrifying a bus fleet. We develop physics-informed machine learning models to evaluate energy consumption, carbon emissions, health impacts, and the total cost of ownership for each transit route. We demonstrate our tool with a case study of the bus lines in the Greater Boston and Milan metropolitan areas. While the capital cost of electric buses and the price of diesel fuel may impact the economic benefit of electrification, we expect significant improvements in health and environmental performances. We estimate about 50% health cost reduction by electrifying only 4 routes out of 15 to 19 routes in each city. For Boston and Milan, some routes are estimated to produce less than 5% of the diesel bus emissions and 14%, respectively, for which investment in electrification can be prioritized."""
output4 = """This research develops a machine learning framework to estimate the economic, environmental, and social values of electrifying public transit buses. Because it is focused on developing a machine learning technique, rather than the impact of machine learning itself, it is in category B. Class: B"""


In [ ]:
## PROMPT GPT-4o mini again

from openai import OpenAI
from dotenv import load_dotenv
import time
import logging
from tqdm import tqdm
import re

# set up logging
logging.basicConfig(level = logging.INFO)

# Load progress from the last run (if any)
try:
    classed_As = pd.read_csv("classed_As.csv")
    processed_DOIs = set(classed_As['DOI'])
    data_dict = classed_As.to_dict('list')
except FileNotFoundError:
    classed_As = pd.DataFrame(columns=['Abstract', 'DOI', 'GPT_response', 'GPT_class', 'Input_tokens', 'Output_tokens', 'reason_stop', 'system_fingerprint'])
    processed_DOIs = set()
    data_dict = {
        'Abstract': [],
        'DOI': [],
        'GPT_response': [],
        'GPT_class': [],
        'Input_tokens': [],
        'Output_tokens': [],
        'reason_stop': [],
        'system_fingerprint': []
    }

# set up api
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()
client.api_key = api_key

# loop through abstracts
for index, row in tqdm(As.iterrows(), desc = 'Processing abstracts'):
    if row['DOI'] in processed_DOIs:
        continue
        
    else:
        if pd.isnull(row['Abstract']):
            print('blank!')
            data_dict['Abstract'].append(row['Abstract'])
            data_dict['DOI'].append(row['DOI'])
            data_dict['GPT_response'].append('')
            data_dict['GPT_class'].append('')
            data_dict['Input_tokens'].append('')
            data_dict['Output_tokens'].append('')
            data_dict['reason_stop'].append('')
            data_dict['system_fingerprint'].append('')
        else:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                max_tokens=300,
                temperature=0,
                seed=2345,
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": example1},
                    {"role": "assistant", "content": output1},
                    {"role": "user", "content": example2},
                    {"role": "assistant", "content": output2},
                    {"role": "user", "content": example3}, 
                    {"role": "assistant", "content": output3},
                    {"role": "user", "content": example4},
                    {"role": "assistant", "content": output4},
                    {"role": "user", "content": example5},
                    {"role": "assistant", "content": output5},
                    {"role": "user", "content": example6},
                    {"role": "assistant", "content": output6},
                    {"role": "user", "content": example7},
                    {"role": "assistant", "content": output7},
                    {"role": "user", "content": example8},
                    {"role": "assistant", "content": output8},
                    {"role": "user", "content": example9},
                    {"role": "assistant", "content": output9},
                    {"role": "user", "content": example10},
                    {"role": "assistant", "content": output10},
                    {"role": "user", "content": example11},
                    {"role": "assistant", "content": output11},
                    {"role": "user", "content": example12},
                    {"role": "assistant", "content": output12},
                    {"role": "user", "content": example13},
                    {"role": "assistant", "content": output13},
                    {"role": "user", "content": example14},
                    {"role": "assistant", "content": output14},
                    {"role": "user", "content": row['Abstract']}
                ]
            )
        
            # store data in df
            data_dict['Abstract'].append(row['Abstract'])
            data_dict['DOI'].append(row['DOI'])
            message = response.choices[0].message.content
            data_dict['GPT_response'].append(message)
            try:
                data_dict['GPT_class'].append(re.search(r'Class:\s*([A-Z])', message).group(1))
            except ValueError:
                data_dict['GPT_class'].append('')
            data_dict['Input_tokens'].append(response.usage.prompt_tokens)
            data_dict['Output_tokens'].append(response.usage.completion_tokens)
            data_dict['reason_stop'].append(response.choices[0].finish_reason)
            data_dict['system_fingerprint'].append(response.system_fingerprint)
    
        # create df out of dict
        classed_As = pd.DataFrame(data_dict)
        
        # save progress, add url to processed url list
        if (index + 1) % 10 == 0:
            try:
                if len(classed_As) - len(pd.read_csv("classed_As.csv")) == 10:
                    new_papers = classed_As.tail(10)
                    new_papers.to_csv("classed_As.csv", index=False, mode='a', header=False)
                    logging.info("Saved progress at iteration {}".format(index + 1))
                else:
                    logging.info("Warning: trying to overwrite with blank at iteration {}".format(index + 1))
                    break
            except FileNotFoundError:
                classed_As.to_csv("classed_As.csv", index=False, header = True)
        
        # add to list of processed urls
        processed_DOIs.add(row['DOI'])

    # # sleep to observe rate limit (500 requests per minute)
    # time.sleep(0.01)

# Display the DataFrame
classed_As = pd.DataFrame(data_dict)
classed_As.to_csv("classed_As.csv", index=False)
classed_As